# PTCB project

##### TODO
- Exploration des transcrits anti-sens : <br>
Finir de setup Sandrine pour utilisation d'IGV (install sur son PC + load bigwig et annotation GENCODE). <br>
Ajouter annotation manquantes "à la main" (custom track d'annotation) sur le locus IgH quand Sandrine nous les aura fourni. <br>
Tester le pipeline UClnrR pour une exploration systématique : http://bioinformaticstools.mayo.edu/research/uclncr-pipeline.


- Kallisto : Quantification des transcrits.



- Trouver le meilleur moyen de vérifier la reproductibilité des réplicats.


- Justifier / vérifier que les comptes sont bien strand-specific : <br>
Soit trouver littérature qui dit que featureCounts, à partir de l'annotation (GENCODE), ne compte que les paires de reads qui ont pour brin originel le brin +/- indiqué pour chaque gène. Trouver également littérature qui justifie pour Kallisto (on sait déjà pour ces deux outils qu'il y a des paramètres de strand-spécificité). <br>
Soit vérifier un cas "à la main", cad trouver dans un dataset un endroit où il y a profondeur de reads sur chaque brin, et alors vérifier que les comptes sont bons.


- QC step after mapping : <br>
rseqc (http://rseqc.sourceforge.net/). <br>
Launch MultiQC on STAR "log files".


- Justifier l'utilisation de DESeq2 pour les cas sans réplicats : Pourquoi lui plutot que par exemple EdgeR ?


- Comparaison avec résultats de Nice.

##### Notes

Report :
http://unil.im/Ly33D

Normalization :
http://unil.im/kdTj5

FeatureCounts tests :
http://unil.im/2sAq6

Read counts step by step :
http://unil.im/Ic1Wc

Les souris utilisés par Limoges sont issus d'un "croisement" C57BL/6J avec SV129. La seule chose certaines au vue de la manipulation est que le locus IgH est 100% SV129. Pour le reste c'est hétérogène. Et rappel : Input RNA-seq <=> pool de 3 individus (ayant donc des génomes proches mais de différentes mosaïques C57BL/6J - SV129).

Pefanis : Correspond à du stimulé.

Réplicats (incoming) : 1 R "Eµ FL KO" (replicat de Yves EU), 1 R MAR Ko (replicat de Yves) & 2 R cEµ KO.

Histoire de recomb/transloc random qui peuvent générer des reads en anti-sens sur le génome => need de prendre le BAM, de soustraire tous les reads qui sont sur un locus : "gene pos pos strand" connu, puis de regarder ce qu'il reste?

Ce sont des souris mâles pour toutes les manips EP et YD.

## 1) Data

### 1.1) Raw reads

#### 1.1.1) CRIBL data

Eric : 3B4, 3pRR, DEL, IRIS, PAL, WT. <br>
Yves : 3pRR, EU, MAR, WT. <br>

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;

In [ ]:
### CODE ###
# Stored @Sophia Antipolis
axel -q http://www.genomique.info/data/public/000-export/151109_pinaud.tar;
axel -q http://www.genomique.info/data/public/000-export/DEL_IRIS_fastq.tar;
axel -q http://www.genomique.info/data/public/000-export/160627_denizot.tar;
tar -xvf *.tar;

And some files renaming (for which I lost the script).

#### 1.1.2) Pefanis data

Retrieve all "BC" samples : https://www.ncbi.nlm.nih.gov/sra?term=SRP042355 :

In [ ]:
### CODE ###
for id in 'SRR1302208' 'SRR1302283' 'SRR1302414' \
           'SRR1302785' 'SRR1302786' 'SRR1302787' \
           'SRR1302788' 'SRR1302789' 'SRR1984164' \
           'SRR1984166' 'SRR1984167' 'SRR1984178';
do fastq-dump -O . --split-files $id;
done;

### 1.2) Mus musculus

#### 1.2.1) Reference genome & associated GFF

Downloaded from the "Genome Reference Consortium", latest realease = GRCm38.p5 :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;
cd ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;

In [ ]:
### CODE ###
url='ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/GCA_000001635.7_GRCm38.p5';
axel -q $url/GCA_000001635.7_GRCm38.p5_genomic.fna.gz;
wget -nv $url/GCA_000001635.7_GRCm38.p5_genomic.gff.gz;
unpigz *_GRCm38.p5_genomic*;

Check if they contains the same sequences names :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | sort -u > fna.txt
sed -i 's/>//' fna.txt 
awk -F "[[:space:]]" '{print $1}' fna.txt > fna_final.txt
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | sort -u > gff.txt
diff fna_final.txt gff.txt

If the diff command produce no output, it's okay. <br>
One last thing, to avoid potential incompatilities in downstream analysis :

In [ ]:
### CODE ###
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

What we have just shown is that sequences headers in the FNA file are more verbose than the associated name for the same sequence in the GFF file. Given that this extra info is not critical, let's discard it & check that we did this successfully :

In [ ]:
### CODE ###
gawk -i inplace -F "[[:space:]]" '{if($0 ~ /^>/) $0 = $1; print}' GCA_000001635.7_GRCm38.p5_genomic.fna;
grep '^>' GCA_000001635.7_GRCm38.p5_genomic.fna | head -2;
awk -F "\t" '{if($0 !~ /^#/) print $1}' GCA_000001635.7_GRCm38.p5_genomic.gff | head -2;

Load stuff on Genotoul :

In [ ]:
### CODE ###
sudo scp *_GRCm38.p5_genomic* escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mus_musculus;

Remove useless files & compress the remaining for local storage :

In [ ]:
### CODE ###
rm *.txt;
pigz *_GRCm38.p5_genomic*;

#### 1.2.2) Reference transcriptome

Which refererence transcriptome to use with Kallisto ? <br>

Asked question : I plan to work with Kallisto on the mus musculus transcriptome : Which DB should I use to retrieve the transcriptome ? : Genbank, Ensembl or UCSC ? <br>

Answer (Kallisto authors) : "Anyway, our links would simply point you to the ensembl transcriptome when it exists. We typically use this as it is fairly complete (by comparison with others) and the transcript to gene name mapping is pretty nice (also by comparison with others)." <br>

Thus we retrieve transcriptome from Ensembl latest release and use cdna + ncrna (to cover almost as many genes as the GENCODE annotation used with featureCounts). <br>

In [ ]:
### CODE ###
mkdir -p /media/sf_raid/Data/PTCB/mm10_transcriptome;
cd /media/sf_raid/Data/PTCB/mm10_transcriptome;

In [ ]:
### CODE ###
url='ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus';
axel -q $url/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz;
axel -q $url/ncrna/Mus_musculus.GRCm38.ncrna.fa.gz;
cat Mus_musculus.GRCm38.cdna.all.fa.gz \
    Mus_musculus.GRCm38.ncrna.fa.gz \
    > mus_musculus_ensembl_grcm38_r90_cdna_and_ncrna.fasta.gz;

Load stuff on Genotoul :

In [ ]:
### CODE ###
sudo scp mus_musculus_ensembl_grcm38_r90_cdna_and_ncrna.fasta.gz \
     hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/mm10_transcriptome;

#### 1.2.3) GTF annotations

For future use with featureCounts.<br>

Downloaded from "GENCODE" (for GRCm38.p5 genome). <br>
"It contains the comprehensive gene annotation on the reference chromosomes, scaffolds, assembly patches and alternate loci (haplotypes)". <br>
More precisely, it annotate a lot of gene/transcript biotypes : https://www.gencodegenes.org/gencode_biotypes.html.

In [ ]:
### CODE ###
axel -q ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_mouse/\
release_M14/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf.gz;

#### 1.2.4) Ensembl vs GENCODE

We want at some point to compare results obtained with the "genomic path", i.e. STAR + featureCounts + DESeq2 vs results from the "transcriptomic path", i.e. Kallisto + DESeq2. Thus, we need make sure that featureCounts (using GENCODE annotations) and Kallisto (using transcriptomes sequences) will compute reads counts at the gene level for approximately the same genes. <br>

##### Get Ensembl genes name list contained in GENCODE gtf : 

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Data/Mus_musculus;
unpigz gencode.vM14.chr_patch_hapl_scaff.annotation.gtf.gz;
awk -F " " '{if($0 !~ /^#/) print $10}' gencode.vM14.chr_patch_hapl_scaff.annotation.gtf | sort -u > gencode_gene_ids_u.txt;
sed -i 's/;//g' gencode_gene_ids_u.txt;
sed -i 's/\"//g' gencode_gene_ids_u.txt;
awk -F '\t' -v OFS='\t' '{sub(/\.[0-9]*/, "", $1)} 1' gencode_gene_ids_u.txt > gencode_gene_ids_u_proper.txt;
wc -l gencode_gene_ids_u_proper.txt;
# 52366 gencode_gene_ids_u_proper.txt
rm gencode_gene_ids_u.txt;
pigz gencode.vM14.chr_patch_hapl_scaff.annotation.gtf;
cp gencode_gene_ids_u_proper.txt ~/Desktop/PTCB/BISCEm/Ensembl_transcriptome_vs_gencode;

##### Get Ensembl genes name list contained in our Ensembl transcriptome :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Ensembl_transcriptome_vs_gencode;
# DL
axel -q ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus/cdna/Mus_musculus.GRCm38.cdna.all.fa.gz;
axel -q ftp://ftp.ensembl.org/pub/release-90/fasta/mus_musculus/ncrna/Mus_musculus.GRCm38.ncrna.fa.gz;
# Grep non-fasta lines
zgrep '^>' Mus_musculus.GRCm38.cdna.all.fa.gz > headers_cdna.txt;
zgrep '^>' Mus_musculus.GRCm38.ncrna.fa.gz > headers_ncrna.txt;
# Count number of transcripts
for f in *.txt;
do printf $f"\t";
   sort -u $f | wc -l;
done;
# headers_cdna.txt    109282
# headers_ncrna.txt    19932
# Clear formatting -> ensembl_gene_name
awk -F " " '{print $4}' headers_cdna.txt > genes_cdna.txt;
awk -F " " '{print $4}' headers_ncrna.txt > genes_ncrna.txt;
sort -u genes_cdna.txt > genes_cdna_u.txt;
sort -u genes_ncrna.txt > genes_ncrna_u.txt;
awk -F "\t" '{sub(/\.[0-9]*/, "", $1)} 1' genes_cdna_u.txt > genes_cdna_u_v2.txt;
awk -F "\t" '{sub(/\.[0-9]*/, "", $1)} 1' genes_ncrna_u.txt > genes_ncrna_u_v2.txt;
awk -F "\t" '{sub(/gene:/, "", $1)} 1' genes_cdna_u_v2.txt > genes_cdna_u_v3.txt;
awk -F "\t" '{sub(/gene:/, "", $1)} 1' genes_ncrna_u_v2.txt > genes_ncrna_u_v3.txt;
wc -l *_v3.txt;
# 34839 genes_cdna_u_v3.txt
# 15220 genes_ncrna_u_v3.txt
# 50059 total
cat genes_cdna_u_v3.txt genes_ncrna_u_v3.txt > genes_cdna_and_ncrna.txt;
sort -u genes_cdna_and_ncrna.txt > genes_cdna_and_ncrna_u.txt;
wc -l genes_cdna_and_ncrna_u.txt;
# 50059 genes_cdna_and_ncrna_u.txt;
rm !(gencode_gene_ids_u_proper.txt|genes_cdna_and_ncrna_u.txt);

##### Finally, check their intersection :

In [ ]:
### CODE ###
comm gencode_gene_ids_u_proper.txt genes_cdna_and_ncrna_u.txt > comm_gencode_vs_ensembl.tsv;
printf "unique to gencode :\t"; awk -F "\t" '{if($1 != "") print $1}' comm_gencode_vs_ensembl.tsv | wc -l;
printf "unique to ensembl :\t"; awk -F "\t" '{if($2 != "") print $2}' comm_gencode_vs_ensembl.tsv | wc -l;
printf "common to both :\t"; awk -F "\t" '{if($3 != "") print $3}' comm_gencode_vs_ensembl.tsv | wc -l;
# unique to gencode :    3047
# unique to ensembl :    740
# common to both :    49319

49319 genes are common (92.87%), so we consider that we can compare DE results with counts coming from featureCounts and from Kallisto.

## 2) Raw reads QC & filtering

### 2.1) FastQC

#### 2.1.1) CRIBL data

Setup working directory & paths :

In [ ]:
### CODE ###
reads_dir='/home/erwann/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice';
cd $reads_dir;
mkdir -p $reads_dir/QC;
mkdir -p ~/Software/FastQC/Outputs/PTCB;

Launch FastQC :

In [ ]:
### CODE ###
for f in *.fastq.gz;
do echo $f; fastqc --outdir ~/Software/FastQC/Outputs/PTCB -f fastq $f;
done >> $reads_dir/QC/fastqc.verbose;

#### 2.1.2) Pefanis data

The exact same thing, just change file and directory names

### 2.2) MultiQC

#### 2.2.1) CRIBL data

We compile FastqQC analysis for all the data & at the sequencing-run level (EP & YD), to highlight potential run effect.

In [ ]:
### CODE ###
fastqc_dir='/home/erwann/Software/FastQC/Outputs/PTCB';
mkdir -p $reads_dir/QC;
multiqc $fastqc_dir \
        -n ptcb \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb.verbose 2>&1;
multiqc $fastqc_dir/*_EP_fastqc* \
        -n ptcb_ep \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_ep.verbose 2>&1;
multiqc $fastqc_dir/*_YD_fastqc* \
        -n ptcb_yd \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb_yd.verbose 2>&1;

Visualize QC results :

In [ ]:
### CODE ###
chromium-browser $reads_dir/QC/*.html;

Conclusions :
Nice data overall. We can be a bit stringent during filtering to keep HQ reads.

#### 2.2.2) Pefanis data

The exact same thing, just change file and directory names

### 2.3) Filtering

Trimmomatic 0.36

For both data CRIBL & Pefanis, FastQC detected Illumina 1.9 encoding. Referring to https://en.wikipedia.org/wiki/FASTQ_format, Illumina 1.8+ => Phred+33.

Sequences are trimmed with $TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar.
Given MultiQC output, we trim starting and ending bases if their quality is < 25. We then trim with a sliding window of 5 bases, cutting the read at those bases if the mean quality drops below 25. Finally, we discard sequence < 50 nts.

#### 2.3.1) CRIBL data

In [ ]:
### CODE ###
$TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar
cd ~/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice;
mkdir -p ../Reads_raw_Nice_trim;
for id in '3B4_R_1_EP' '3B4_S_1_EP' '3pRR_R_1_EP' '3pRR_S_1_EP' \
          'DEL_R_1_EP' 'DEL_S_1_EP' 'IRIS_R_1_EP' 'IRIS_S_1_EP' \
          'PAL_R_1_EP' 'PAL_S_1_EP' 'WT_R_1_EP' 'WT_S_1_EP';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_EP.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_EP_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_EP.verbose;

for id in '3pRR_R_1_YD' '3pRR_S_1_YD' 'EU_R_1_YD' 'EU_S_1_YD' \
          'MAR_R_1_YD' 'MAR_S_1_YD' 'WT_R_1_YD' 'WT_S_1_YD';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id.fastq.gz \
                      ${id%_*_*}'_2_YD.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/$id'_unpaired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_paired.fastq.gz' \
                      ../Reads_raw_Nice_trim/${id%_*_*}'_2_YD_unpaired.fastq.gz' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_YD.verbose;

#### 2.3.2) Pefanis data

In [ ]:
### CODE ###
$TRIM = ~/Software/Trimmomatic_0.36/trimmomatic-0.36.jar
cd /media/raid/Data/PTCB/Pefanis;
mkdir -p Trimmomatic;
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' \
          'SRR1302788' 'SRR1302789';
do java -jar $TRIM PE -threads 8 \
                      -phred33 \
                      $id'_1.fastq' \
                      $id'_2.fastq' \
                      ./Trimmomatic/$id'_1.paired.fastq' \
                      ./Trimmomatic/$id'_1.unpaired.fastq' \
                      ./Trimmomatic/$id'_2.paired.fastq' \
                      ./Trimmomatic/$id'_2.unpaired.fastq' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_PE.verbose;
for id in 'SRR1302414' 'SRR1302785';
do java -jar $TRIM SE -threads 8 \
                      -phred33 \
                      $id'_1.fastq' \
                      ./Trimmomatic/$id'_1.paired.fastq' \
                      ./Trimmomatic/$id'_1.unpaired.fastq' \
                      LEADING:25 TRAILING:25 SLIDINGWINDOW:5:25 MINLEN:50;
done &>> trim_SE.verbose;

### 2.4) QC post filtering

In [ ]:
### CODE ###
reads_dir='/home/erwann/Desktop/PTCB/BISCEm/Data/Reads_raw_Nice_trim/';
mkdir -p $reads_dir/QC;
mkdir -p ~/Software/FastQC/Outputs/PTCB/Trim;
for f in *.fastq.gz;
do echo $f; fastqc --outdir ~/Software/FastQC/Outputs/PTCB/Trim -f fastq $f;
done >> $reads_dir/QC/fastqc.verbose;

In [ ]:
### CODE ###
fastqc_dir='/home/erwann/Software/FastQC/Outputs/PTCB/Trim';
multiqc $fastqc_dir \
        -n ptcb_trim \
        -o $reads_dir/QC \
        -m fastqc -f > $reads_dir/QC/multiQC_ptcb.verbose 2>&1;


## 3) Mapping

### 3.1) Build index :

#### 3.1.1) STAR :

On Genotoul via STAR, version 2.5.2b :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=12G
#$ -N star_idx
#$ -o /home/escaon/work/PTCB/star_idx.o
#$ -e /home/escaon/work/PTCB/star_idx.e

work='/home/escaon/work/PTCB/Mus_musculus';
star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR'
nt=8;

$star --runThreadN $nt \
      --runMode genomeGenerate \
      --genomeDir $work \
      --genomeFastaFiles $work/GCA_000001635.7_GRCm38.p5_genomic.fna \
      --sjdbGTFfile $work/GCA_000001635.7_GRCm38.p5_genomic.gff \
      --sjdbGTFtagExonParentTranscript Parent \
      --sjdbOverhang 150;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_idx.sh;

#### 3.1.2) Kallisto :

On Genotoul via Kallisto, version 0.43.0 :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=120G

#####Index KALLISTO
kallisto index --make-unique -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.fa

In [ ]:
### CODE ###
qsub simple_index_kallisto.sh;

### 3.2) Map reads :

###### 3.2.1) CRIBL data

First script : star_3B4_R_EP.sh

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=10G
#$ -N star_3B4_R_EP
#$ -o /home/escaon/work/PTCB/star_3B4_R_EP.o
#$ -e /home/escaon/work/PTCB/star_3B4_R_EP.e

star='/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR';
save='/home/escaon/save/PTCB/Reads_raw_Nice_trim';
work='/home/escaon/work/PTCB';
nt=8;

for id in '3B4_R_EP';
do $star --runThreadN $nt \
         --genomeDir $work/Mus_musculus \
         --readFilesIn $save/${id%_*}'_1_EP_paired.fastq.gz' $save/${id%_*}'_2_EP_paired.fastq.gz' \
         --readFilesCommand zcat \
         --outFileNamePrefix $work/Mapped_vs_GRCm38.p5/$id'_' \
         --outSAMtype BAM SortedByCoordinate;
done;

In [ ]:
### CODE ###
qsub -pe parallel_smp 8 star_3B4_R_EP.sh;

Duplicate this script for others EP datasets & launch jobs :

In [ ]:
### CODE ###
for id in '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' \
          'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' \
          'PAL_S_EP' 'WT_R_EP' 'WT_S_EP';
do  cp star_3B4_R_EP.sh star_$id.sh;
    sed -i "s/3B4_R_EP/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

Create a base script for YD, duplicate it & launch jobs :

In [ ]:
### CODE ###
cp star_WT_R_EP.sh star_WT_R_YD.sh;
grep --color 'EP' star_WT_R_YD.sh; # check if we are not going to destroy anything
sed -i "s/EP/YD/" star_WT_R_YD.sh;
qsub -pe parallel_smp 8 star_WT_R_YD.sh;

for id in '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_S_YD';
do  cp star_WT_R_YD.sh star_$id.sh;
    sed -i "s/WT_R_YD/$id/" star_$id.sh;
    qsub -pe parallel_smp 8 star_$id.sh;
done;

###### 3.2.2) Pefanis data

###### Paired end :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment STAR PE
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do
/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR --genomeDir /home/hbastien/work/PTCB/Mus_musculus/ --readFilesIn /home/hbastien/work/PTCB/Pefanis/'Trimmed_paired_'$id'_1.fastq' /home/hbastien/work/PTCB/Pefanis/'Trimmed_paired_'$id'_2.fastq' --outFileNamePrefix Experiment_PE_Star$id
done

In [ ]:
### CODE ###
qsub manageStarPE.sh

###### Single end :

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment STAR SE
for id in 'SRR1302414' 'SRR1302785';
do
/usr/local/bioinfo/src/STAR/STAR-2.5.2b/bin/Linux_x86_64_static/STAR --genomeDir /home/hbastien/work/PTCB/Mus_musculus/ --readFilesIn /home/hbastien/work/PTCB/Pefanis/'Trimmed_'$id'_1.fastq' --outFileNamePrefix Experiment_SE_Star$id
done

In [ ]:
### CODE ###
qsub manageStarSE.sh

### 3.3) Index BAM

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -M erwan.scaon@unilim.fr
#$ -m bea
#$ -l mem=40G
#$ -l h_vmem=48G
#$ -N samtools_idx
#$ -o /home/escaon/work/PTCB/samtools_idx.o
#$ -e /home/escaon/work/PTCB/samtools_idx.e

work='/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5';

cd $work;
for f in *.bam;
do samtools index $f;
done;

In [ ]:
### CODE ###
qsub samtools_idx.sh;

### 3.4) RNA-seq alignment QC

http://rseqc.sourceforge.net

Conseils de Steph pour reseqc : <br>
Salut,
alors:
t'as déjà les stats d'alignements avec bam_stat 
infer_experiment.py qui te permet de déterminer le protocole utilisé pour la prépa de ta librairie (ce qui t'oriente pour feature counts !)
geneBody_coverage où tu vois le profil de couverture sur le gène : à analyser avec tous les échantillons pour voir si y en a un qui déconne !
inner_distance : normalement centré sur la taille de ta librairie 

Ensuite c'est des outils d'évaluation: pour donner de l'info sur ton alignement et en gardant à l'esprit que çà peut te permettre de voir si échantillon déconne:
read_distribution
junction_annotation 

Et puis évaluer la quantité de duplicat de PCR (read_duplication) avant et après retrait avec samtools rmdup

Voilà ce que j'avais utilisé en formation  (+ aussi la saturation des jonctions et la saturation de la librairie mais c'est peut-être un peu moins parlant comme info)

## 4) Counts

##### Retrieve BAMs locally :

In [ ]:
### CODE ###
mkdir -p ~/Desktop/PTCB/BISCEm/Outputs;
sudo scp -r escaon@genotoul.toulouse.inra.fr:/home/escaon/work/PTCB/Mapped_vs_GRCm38.p5 ~/Desktop/PTCB/BISCEm/Outputs;
sudo scp -r hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/Mapped_vs_GRCm38.p5 /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Star

Before starting the counts, we need to create a "chr.aliases" file which link chrs names in GRCm38.p5 with chrs names in GENCODE annotations. This file was created from this assembly report :

In [ ]:
### CODE ###
wget -q ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/635/\
GCA_000001635.7_GRCm38.p5/GCA_000001635.7_GRCm38.p5_assembly_report.txt;

### 4.1) FeatureCounts

Count with featureCounts (Rsubread 1.24.2) :

#### 4.1.1) CRIBL data

In [ ]:
### CODE ###
path='/home/erwann/Desktop/PTCB/BISCEm';
cd $path/Outputs/Star;
for id in '3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' \
          'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' \
          'PAL_R_EP' 'PAL_S_EP' 'WT_R_EP' 'WT_S_EP' \
          '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' \
          'MAR_R_YD' 'MAR_S_YD' 'WT_R_YD' 'WT_S_YD';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('../FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)

fc = featureCounts(files=in_file,
                   annot.ext="../../Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="../../Data/Mus_musculus/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

#### 4.2.2) Pefanis data

##### a) Paired end

In [ ]:
### CODE ###
path='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $path/Outputs/Star;
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)
print(paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''))
fc = featureCounts(files=paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''),
                   annot.ext="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=TRUE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=2,
                   requireBothEndsMapped=TRUE,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

##### b) Single end

In [ ]:
### CODE ###
path='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $path/Outputs/Star;
for id in 'SRR1302414' 'SRR1302785';
do Rscript $path/Scripts/featureCounts.R $id;
done &>> $path/Outputs/FeatureCounts/featureCounts.verbose;

Content of featureCounts.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

args = commandArgs(trailingOnly=TRUE)
in_file = paste(args[1], 'bam', sep = '.')
out_file = paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/FeatureCounts/', args[1], '.tsv', sep = '')

library(Rsubread)
print(paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''))
fc = featureCounts(files=paste('/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Star/', in_file, sep = ''),
                   annot.ext="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Data/Mus_musculus/gencode.vM14.chr_patch_hapl_scaff.annotation.gtf",
                   isGTFAnnotationFile=TRUE,
                   GTF.featureType="exon",
                   GTF.attrType="gene_id",
                   chrAliases="/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/chr.aliases",
                   allowMultiOverlap=FALSE,
                   ignoreDup=FALSE,
                   isPairedEnd=FALSE,
                   nthreads=8,
                   minMQS=0,
                   maxMOp=10,
                   checkFragLength=FALSE,
                   useMetaFeatures=TRUE,
                   strandSpecific=0,
                   countChimericFragments=FALSE,
                   primaryOnly=TRUE)
write.table(fc$counts, out_file, append= T, sep='\t', col.names=NA)

### 4.2) Kallisto

On Genotoul via Kallisto, version 0.43.0 :

#### 4.2.1) CRIBL data

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO
for id in '3B4_R' '3B4_S' '3pRR_R' '3pRR_S' 'DEL_R' 'DEL_S' 'IRIS_R' 'IRIS_S' 'PAL_R' 'PAL_S' 'WT_R' 'WT_S';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id'EP' -b 100 --rf-stranded /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_1_EP_paired.fastq.gz' /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_2_EP_paired.fastq.gz'
done
for id in '3pRR_R' '3pRR_S' 'EU_R' 'EU_S' 'MAR_R' 'MAR_S' 'WT_R' 'WT_S';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id'YD' -b 100 --rf-stranded /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_1_YD_paired.fastq.gz' /home/escaon/save/PTCB/Reads_raw_Nice_trim/$id'_2_YD_paired.fastq.gz'
done

In [ ]:
### CODE ###
qsub manageKallisto.sh

#### 4.2.2) Pefanis data

##### a) Paired end

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO PE
for id in 'SRR1302208' 'SRR1302283' 'SRR1302786' 'SRR1302787' 'SRR1302788' 'SRR1302789';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id -b 100 /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_paired_'$id'_1.fastq' /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_paired_'$id'_2.fastq'
done

In [ ]:
### CODE ###
qsub manageKallistoPE.sh

##### b) Single end

In [ ]:
### CODE ###
#!/bin/bash
#$ -q workq
#$ -m bea
#$ -l mem=8G
#$ -l h_vmem=30G
#

#####Alignment KALLISTO SE
for id in 'SRR1302414' 'SRR1302785';
do
kallisto quant -i /home/hbastien/work/PTCB/mm10_transcriptome/Mus_musculus.GRCm38.cdna.all.idx -o kallisto$id -b 100 --single -l 100 -s 20 /home/hbastien/work/PTCB/Pefanis_rebuild/'Trimmed_'$id'_1.fastq'
done

In [ ]:
### CODE ###
qsub manageKallistoSE.sh

##### Retrieve count files locally :

In [ ]:
### CODE ###
sudo scp -r hbastien@genotoul.toulouse.inra.fr:/home/hbastien/work/PTCB/kallisto* /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Kallisto

### 4.3) Downstream analysis compatibility
Slightly modify featureCounts output to make it compatible with future steps of the analysis.
We want our gene names to be the same than those contained in ensembl when retrieved with biomart.

#### 4.3.1) FeatureCounts

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd ~/Desktop/PTCB/BISCEm/Outputs/FeatureCounts;
for f in *.tsv;
do sed 's/\"//g' $f > ${f%.*}".temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for f in *.temp.tsv;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $f > ${f%%.*}"_ensembl_gene_id.tsv";
done;
rm *.temp.tsv;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for f in *.tsv;
do awk -F '\t' '{print $1}' $f | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

#### 4.3.2) Kallisto

Remove spreadsheet hidden ' " ' for downstream compatility with our awk manipulation :

In [ ]:
### CODE ###
cd /run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB/Outputs/Kallisto;
for d in ./*;
do sed 's/\"//g' $d/"abundance.tsv" > $d/"abundance.temp.tsv";
done;

Use awk sub to convert "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" :

In [ ]:
### CODE ###
for d in ./*;
do awk -F '\t' -v OFS='\t' 'NR > 1 {sub(/\.[0-9]*/, "", $1)} 1' $d/"abundance.tsv" > $d/"abundance_ensembl_gene_id.tsv";
rm $d/*.temp.tsv;
done;

Check that the removal of ".[0-9]?" from the gene id column did not change the number of unique gene ids :

In [ ]:
### CODE ###
for d in ./*;
do awk -F '\t' '{print $1}' $d/"abundance.tsv" | sort -u | wc -l;
done;

We did show that transforming gene name "ENSMUSG00000098104.12" -> "ENSMUSG00000098104" did not change the number unique gene names.

## 5) DE analysis

Perfomed with DESeq2_1.16.1 (using Bioconductor 3.5 (BiocInstaller 1.26.0), R 3.4.1 (2017-06-30)) : <br>

Each analysis (R script) will output a raw TSV of DE genes, a TSV of SDE genes, MA, Volcano & Correlation plots. <br>
For 1 vs 1 comparison, SDE genes are defined as follow : DE genes having : log2(foldchange) absolute value > 2 & p-value < 0.05 & at least 100 counts in one of the two conditions. <br>
For future comparison with replicates, SDE genes will be DE genes with adjusted p-value < 0.05.

##### A quick word regarding comparisions with no replicates

From DESeq2 documentation (https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#can-i-use-deseq2-to-analyze-a-dataset-without-replicates) : <br>
Without replicates, a warning is printed, that the samples are treated as replicates for estimation of dispersion. This kind of analysis is only useful for exploring the data, but will not provide the kind of proper statistical inference on differences between groups. Without biological replicates, it is not possible to estimate the biological variability of each gene.

### 5.1) Resting vs Stimulated for the same mouse model

#### 5.1.1) No replicates

##### With FeatureCounts :

In [1]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR=('3B4_R_EP' '3pRR_R_EP' 'DEL_R_EP' 'IRIS_R_EP' 'PAL_R_EP' 'WT_R_EP' '3pRR_R_YD' 'EU_R_YD' 'MAR_R_YD' 'WT_R_YD')
arrayS=('3B4_S_EP' '3pRR_S_EP' 'DEL_S_EP' 'IRIS_S_EP' 'PAL_S_EP' 'WT_S_EP' '3pRR_S_YD' 'EU_S_YD' 'MAR_S_YD' 'WT_S_YD')

length=${#arrayR[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_no_replicates_fc.R ${arrayR[$i]} ${arrayS[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_no_replicates_fc.verbose;
done;

Content of deseq2_R_vs_S_no_replicates_fc.R :

In [1]:
### CODE ###
#!/usr/bin/env Rscript
args = commandArgs(trailingOnly=TRUE)

string = paste(substr(args[1],1,nchar(args[1])-5),'_',substr(args[1],nchar(args[1])-1,nchar(args[1])), sep="")

# Open featureCounts results for the same mouse model & same run
R = read.table(paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep=""))
S = read.table(paste("./Outputs/FeatureCounts/",args[2],"_ensembl_gene_id.tsv", sep=""))

# Print files names
paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[2],"_ensembl_gene_id.tsv", sep="")

# Create dataframe
df = data.frame(row.names = row.names(R),
                R = R[1],
                S = S[1])
colnames(df) <- c(args[1], args[2])

# Create design
condition <- c('R', 'S')
type <- c('paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(args[1], args[2])

# Check of we have the same colnames in df & design
all(rownames(df_infos) %in% colnames(df))

# Start DESeq2
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = df,
                              colData = df_infos,
                              design = ~ condition)

# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]

# Check on many genes it discarded
summary(dds)

# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('R','S'))

# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)

# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
                 contrast=c("condition","R","S"),
                 res=res)

# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png(paste('./Outputs/DESeq2/Plots/MA/R_vs_S_', string, '_fc_MA.png', sep=""))
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()

# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png(paste('./Outputs/DESeq2/Plots/Volcano/R_vs_S_', string, '_fc_volcano.png', sep=""))
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()

# Retrieve and print sizeFactors for future BAM normalization
dds <- estimateSizeFactors(dds)
sizeFactors(dds)

# Retrieve normalized counts for each condition for future merge
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "R_normalized_count"
colnames(nor_counts_df)[3] <- "S_normalized_count"

# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"

# Add biotypes infos thanks to ensembl annotations using biomart
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)

# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

# Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]


# Save raw results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Raw/R_vs_S_', string, '_all_fc.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

# Filter results to keep only significantly DE genes
res_biotyped_counts <- res_biotyped_counts[(res_biotyped_counts$pvalue<0.05 & (res_biotyped_counts$log2FoldChange<=(-2) | res_biotyped_counts$log2FoldChange>=2) & (res_biotyped_counts[11]>=100 | res_biotyped_counts[12]>=100)),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save filtered results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Filtered/R_vs_S_', string, '_fc.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# New plots to check variability between samples
# See "5.2 The rlog and variance stabilizing transformations" at www.bioconductor.org/help/workflows/rnaseqGene/
library("dplyr")

# log2 on normalized counts
log2nor_df <- as_data_frame(log2(counts(dds, normalized=TRUE)[, 1:2]+1))
log2nor_r <- cor.test(log2nor_df[[colnames(dds)[1]]],
                      log2nor_df[[colnames(dds)[2]]],
                      method = "pearson")

# regularized-logarithm transformation (rlog)
rld <- rlog(dds, blind = FALSE)
rld_df <- as_data_frame(assay(rld)[, 1:2])
rld_r <- cor.test(rld_df[[colnames(dds)[1]]],
                  rld_df[[colnames(dds)[2]]],
                  method = "pearson")

# variance stabilizing transformation (vst)
vsd <- varianceStabilizingTransformation(dds, blind=FALSE)
vsd_df <- as_data_frame(assay(vsd)[, 1:2])
vsd_r <- cor.test(vsd_df[[colnames(dds)[1]]],
                  vsd_df[[colnames(dds)[2]]],
                  method = "pearson")

# Start actual plots
library("ggplot2")

# log2 on normalized counts
colnames(log2nor_df)[1:2] <- c("x", "y")
plot2 <- ggplot(log2nor_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_fc_log2nor.png', sep=""))
plot2 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(log2nor_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# regularized-logarithm transformation (rlog)
colnames(rld_df)[1:2] <- c("x", "y")
plot3 <- ggplot(rld_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_fc_rld.png', sep=""))
plot3 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(rld_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# variance stabilizing transformation (vst)
colnames(vsd_df)[1:2] <- c("x", "y")
plot4 <- ggplot(vsd_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_fc_vsd.png', sep=""))
plot4 + geom_text(x=11,
                  y=15,
                  label=paste("Pearson = ",
                              round(vsd_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

##### With Kallisto :

In [2]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
arrayR=('3B4_R_EP' '3pRR_R_EP' 'DEL_R_EP' 'IRIS_R_EP' 'PAL_R_EP' 'WT_R_EP' '3pRR_R_YD' 'EU_R_YD' 'MAR_R_YD' 'WT_R_YD')
arrayS=('3B4_S_EP' '3pRR_S_EP' 'DEL_S_EP' 'IRIS_S_EP' 'PAL_S_EP' 'WT_S_EP' '3pRR_S_YD' 'EU_S_YD' 'MAR_S_YD' 'WT_S_YD')

length=${#arrayR[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_R_vs_S_no_replicates_ka.R ${arrayR[$i]} ${arrayS[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_R_vs_S_no_replicates_ka.verbose;
done;

Content of deseq2_R_vs_S_no_replicates_ka.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript
args = commandArgs(trailingOnly=TRUE)
string = paste(substr(args[1],1,nchar(args[1])-5),'_',substr(args[1],nchar(args[1])-1,nchar(args[1])), sep="")

# Open featureCounts results for the same mouse model & same run
R = paste("./Outputs/Kallisto/kallisto",args[1],"/abundance_ensembl_gene_id.tsv", sep="")
S = paste("./Outputs/Kallisto/kallisto",args[2],"/abundance_ensembl_gene_id.tsv", sep="")

# Print files names
files <- file.path(c(R, S))
names(files) <- paste0(c(args[1], args[2]))
files

# use biomart to help import Kallisto results (ensembl_transcript_id)
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_transcript_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)

# Do remaining needed stuff to import Kallisto counts into DESeq2
library("tximport")
library("GenomicFeatures")

mmusculusEnsembl <- makeTxDbFromBiomart(dataset="mmusculus_gene_ensembl")

k <- keys(mmusculusEnsembl, keytype = "GENEID")
df <- select(mmusculusEnsembl, keys = k, keytype = "GENEID", columns = "TXNAME")
tx2gene <- df[, 2:1]
txi.kallisto.tsv <- tximport(files, type = "kallisto", tx2gene = tx2gene)
df <- data.frame(txi.kallisto.tsv$counts, check.names = FALSE)

# Create design
condition <- c('R', 'S')
type <- c('paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(args[1],args[2])

# Check of we have the same colnames in df & design
all(rownames(df_infos) %in% colnames(df))

# Start DESeq2
library("DESeq2")
dds <- DESeqDataSetFromTximport(txi.kallisto.tsv,
                                df_infos,
                                ~condition)

# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]

# Check on many genes it discarded
summary(dds)

# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('R','S'))

# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)

# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
                contrast=c("condition","R","S"),
                res=res)

# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png(paste('./Outputs/DESeq2/Plots/MA/R_vs_S_', string, '_ka_MA.png', sep=""))
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()

# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png(paste('./Outputs/DESeq2/Plots/Volcano/R_vs_S_', string, '_ka_volcano.png', sep=""))

plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")

sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()

# Retrieve and print sizeFactors for future BAM normalization
dds <- estimateSizeFactors(dds)
sizeFactors(dds)

# Retrieve normalized counts for each condition for future merge
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "R_normalized_count"
colnames(nor_counts_df)[3] <- "S_normalized_count"

# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"

# Add biotypes infos thanks to ensembl annotations using biomart (ensembl_gene_id)
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

# Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]

write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Raw/R_vs_S_', string, '_all_ka.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

# Filter results to keep only significantly DE genes
res_biotyped_counts <- res_biotyped_counts[(res_biotyped_counts$pvalue<0.05 & (res_biotyped_counts$log2FoldChange<=(-2) | res_biotyped_counts$log2FoldChange>=2) & (res_biotyped_counts[11]>=100 | res_biotyped_counts[12]>=100)),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save filtered results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Filtered/R_vs_S_', string, '_ka.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# New plots to check variability between samples
# See "5.2 The rlog and variance stabilizing transformations" at www.bioconductor.org/help/workflows/rnaseqGene/
library("dplyr")

# log2 on normalized counts
log2nor_df <- as_data_frame(log2(counts(dds, normalized=TRUE)[, 1:2]+1))
log2nor_r <- cor.test(log2nor_df[[colnames(dds)[1]]],
                      log2nor_df[[colnames(dds)[2]]],
                      method = "pearson")

# regularized-logarithm transformation (rlog)
rld <- rlog(dds, blind = FALSE)
rld_df <- as_data_frame(assay(rld)[, 1:2])
rld_r <- cor.test(rld_df[[colnames(dds)[1]]],
                  rld_df[[colnames(dds)[2]]],
                  method = "pearson")

# variance stabilizing transformation (vst)
vsd <- varianceStabilizingTransformation(dds, blind=FALSE)
vsd_df <- as_data_frame(assay(vsd)[, 1:2])
vsd_r <- cor.test(vsd_df[[colnames(dds)[1]]],
                  vsd_df[[colnames(dds)[2]]],
                  method = "pearson")

# Start actual plots
library("ggplot2")

# log2 on normalized counts
colnames(log2nor_df)[1:2] <- c("x", "y")
plot2 <- ggplot(log2nor_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_ka_log2nor.png', sep=""))
plot2 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(log2nor_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# regularized-logarithm transformation (rlog)
colnames(rld_df)[1:2] <- c("x", "y")
plot3 <- ggplot(rld_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_ka_rld.png', sep=""))
plot3 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(rld_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# variance stabilizing transformation (vst)
colnames(vsd_df)[1:2] <- c("x", "y")
plot4 <- ggplot(vsd_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/R_vs_S_', string, '_ka_vsd.png', sep=""))
plot4 + geom_text(x=11,
                  y=15,
                  label=paste("Pearson = ",
                              round(vsd_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

### 5.2) Mutant vs WT (for R and for S)

#### 5.2.1) No replicates

##### 5.2.2.1) CRIBL

##### With FeatureCounts :

In [5]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_no_replicates_fc.R ${array[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_no_replicates_fc.verbose;
done;

Content of deseq2_mutant_vs_wt_no_replicates_fc.R :

In [2]:
### CODE ###
#!/usr/bin/env Rscript
args = commandArgs(trailingOnly=TRUE)

#args=c('3B4_R_EP')

string = substr(args[1], nchar(args[1])-4,nchar(args[1]))
string2 = paste(substr(args[1],1,nchar(args[1])-5),'_vs_WT_',substr(args[1],nchar(args[1])-3,nchar(args[1])), sep="")

# Open featureCounts results for WT & Mutant for the same condition (R or S) 
WT = read.table(paste("./Outputs/FeatureCounts/WT",string,"_ensembl_gene_id.tsv", sep=""))
KO = read.table(paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep=""))

# Print files names
paste("./Outputs/FeatureCounts/WT",string,"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep="")

# Create dataframe
df = data.frame(row.names = row.names(WT),
                WT = WT[1],
                KO = KO[1])
colnames(df) <- c(paste("WT",string, sep=""), args[1])

# Create design
condition <- c('WT', 'KO')
type <- c('paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(paste("WT",string, sep=""), args[1])

# Check of we have the same colnames in df & design
all(rownames(df_infos) %in% colnames(df))

# Start DESeq2
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = df,
                              colData = df_infos,
                              design = ~ condition)

# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]

# Check on many genes it discarded
summary(dds)

# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('WT','KO'))

# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)

# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
                 contrast=c("condition","WT","KO"),
                 res=res)

# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png(paste('./Outputs/DESeq2/Plots/MA/', string2, '_fc_MA.png', sep=""))
plotMA(res,
       ylim=c(min(res$log2FoldChange),
       max(res$log2FoldChange)))
dev.off()

# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png(paste('./Outputs/DESeq2/Plots/Volcano/', string2, '_fc_volcano.png', sep=""))
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()

# Retrieve and print sizeFactors for future BAM normalization
dds <- estimateSizeFactors(dds)
sizeFactors(dds)

# Retrieve normalized counts for each condition for future merge
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT_normalized_count"
colnames(nor_counts_df)[3] <- "KO_normalized_count"

# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"

# Add biotypes infos thanks to ensembl annotations using biomart
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)

# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

# Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]

# Save raw results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Raw/', string2, '_all_fc.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

# Filter results to keep only significantly DE genes
res_biotyped_counts <- res_biotyped_counts[(res_biotyped_counts$pvalue<0.05 & (res_biotyped_counts$log2FoldChange<=(-2) | res_biotyped_counts$log2FoldChange>=2) & (res_biotyped_counts[11]>=100 | res_biotyped_counts[12]>=100)),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save filtered results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Filtered/', string2, '_fc.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# New plots to check variability between samples
# See "5.2 The rlog and variance stabilizing transformations" at www.bioconductor.org/help/workflows/rnaseqGene/
library("dplyr")

# log2 on normalized counts
log2nor_df <- as_data_frame(log2(counts(dds, normalized=TRUE)[, 1:2]+1))
log2nor_r <- cor.test(log2nor_df[[colnames(dds)[1]]],
                      log2nor_df[[colnames(dds)[2]]],
                      method = "pearson")

# regularized-logarithm transformation (rlog)
rld <- rlog(dds, blind = FALSE)
rld_df <- as_data_frame(assay(rld)[, 1:2])
rld_r <- cor.test(rld_df[[colnames(dds)[1]]],
                  rld_df[[colnames(dds)[2]]],
                  method = "pearson")

# variance stabilizing transformation (vst)
vsd <- varianceStabilizingTransformation(dds, blind=FALSE)
vsd_df <- as_data_frame(assay(vsd)[, 1:2])
vsd_r <- cor.test(vsd_df[[colnames(dds)[1]]],
                  vsd_df[[colnames(dds)[2]]],
                  method = "pearson")

# Start actual plots
library("ggplot2")

# log2 on normalized counts
colnames(log2nor_df)[1:2] <- c("x", "y")
plot2 <- ggplot(log2nor_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_fc_log2nor.png', sep=""))
plot2 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(log2nor_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# regularized-logarithm transformation (rlog)
colnames(rld_df)[1:2] <- c("x", "y")
plot3 <- ggplot(rld_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_fc_rld.png', sep=""))
plot3 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(rld_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# variance stabilizing transformation (vst)
colnames(vsd_df)[1:2] <- c("x", "y")
plot4 <- ggplot(vsd_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_fc_vsd.png', sep=""))
plot4 + geom_text(x=11,
                  y=15,
                  label=paste("Pearson = ",
                              round(vsd_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

##### With Kallisto :

In [6]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
array=('3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD')

length=${#array[@]}
for ((i=0;i<=$length-1;i++));
do
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_no_replicates_ka.R ${array[$i]} &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_no_replicates_ka.verbose;
done;

Content of deseq2_mutant_vs_wt_no_replicates_ka.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript
args = commandArgs(trailingOnly=TRUE)

#args=c('3B4_R_EP')

string = substr(args[1], nchar(args[1])-4,nchar(args[1]))
string2 = paste(substr(args[1],1,nchar(args[1])-5),'_vs_WT_',substr(args[1],nchar(args[1])-3,nchar(args[1])), sep="")

# Open kallisto results for WT & Mutant for the same condition (R or S) 
WT = paste("./Outputs/Kallisto/kallistoWT",string,"/abundance_ensembl_gene_id.tsv", sep="")
KO = paste("./Outputs/Kallisto/kallisto",args[1],"/abundance_ensembl_gene_id.tsv", sep="")

# Print files names
files <- file.path(c(WT, KO))
names(files) <- paste0(c(paste0("WT",string), args[1]))
files

# use biomart to help import Kallisto results (ensembl_transcript_id)
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_transcript_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)

# Do remaining needed stuff to import Kallisto counts into DESeq2
library("tximport")
library("GenomicFeatures")

mmusculusEnsembl <- makeTxDbFromBiomart(dataset="mmusculus_gene_ensembl")

k <- keys(mmusculusEnsembl, keytype = "GENEID")
df <- select(mmusculusEnsembl, keys = k, keytype = "GENEID", columns = "TXNAME")
tx2gene <- df[, 2:1]
txi.kallisto.tsv <- tximport(files, type = "kallisto", tx2gene = tx2gene)
df <- data.frame(txi.kallisto.tsv$counts, check.names = FALSE)

# Create design
condition <- c('WT', 'KO')
type <- c('paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(paste("WT",string, sep=""), args[1])

# Check of we have the same colnames in df & design
all(rownames(df_infos) %in% colnames(df))

# Start DESeq2
library("DESeq2")
dds <- DESeqDataSetFromTximport(txi.kallisto.tsv,
                                df_infos,
                                ~condition) 

# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]

# Check on many genes it discarded
summary(dds)

# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('WT','KO'))

# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)

# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
                contrast=c("condition","WT","KO"),
                res=res)

# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png(paste('./Outputs/DESeq2/Plots/MA/', string2, '_ka_MA.png', sep=""))
plotMA(res,
       ylim=c(min(res$log2FoldChange),
       max(res$log2FoldChange)))
dev.off()

# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png(paste('./Outputs/DESeq2/Plots/Volcano/', string2, '_ka_volcano.png', sep=""))
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")
sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()

# Retrieve and print sizeFactors for future BAM normalization
dds <- estimateSizeFactors(dds)
sizeFactors(dds)

# Retrieve normalized counts for each condition for future merge
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]
colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT_normalized_count"
colnames(nor_counts_df)[3] <- "KO_normalized_count"

# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"

# Add biotypes infos thanks to ensembl annotations using biomart (ensembl_gene_id)
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                         dataset = "mmusculus_gene_ensembl",
                         host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)

# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

# Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]

# Save raw results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Raw/', string2, '_all_ka.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

# Filter results to keep only significantly DE genes
res_biotyped_counts <- res_biotyped_counts[(res_biotyped_counts$pvalue<0.05 & (res_biotyped_counts$log2FoldChange<=(-2) | res_biotyped_counts$log2FoldChange>=2) & (res_biotyped_counts[11]>=100 | res_biotyped_counts[12]>=100)),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save filtered results in a TSV
write.table(res_biotyped_counts,
            paste('./Outputs/DESeq2/Filtered/', string2, '_ka.tsv', sep=""),
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# New plots to check variability between samples
# See "5.2 The rlog and variance stabilizing transformations" at www.bioconductor.org/help/workflows/rnaseqGene/
library("dplyr")

# log2 on normalized counts
log2nor_df <- as_data_frame(log2(counts(dds, normalized=TRUE)[, 1:2]+1))
log2nor_r <- cor.test(log2nor_df[[colnames(dds)[1]]],
                      log2nor_df[[colnames(dds)[2]]],
                      method = "pearson")

# regularized-logarithm transformation (rlog)
rld <- rlog(dds, blind = FALSE)
rld_df <- as_data_frame(assay(rld)[, 1:2])
rld_r <- cor.test(rld_df[[colnames(dds)[1]]],
                  rld_df[[colnames(dds)[2]]],
                  method = "pearson")

# variance stabilizing transformation (vst)
vsd <- varianceStabilizingTransformation(dds, blind=FALSE)
vsd_df <- as_data_frame(assay(vsd)[, 1:2])
vsd_r <- cor.test(vsd_df[[colnames(dds)[1]]],
                  vsd_df[[colnames(dds)[2]]],
                  method = "pearson")

# Start actual plots
library("ggplot2")

# log2 on normalized counts
colnames(log2nor_df)[1:2] <- c("x", "y")
plot2 <- ggplot(log2nor_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_ka_log2nor.png', sep=""))
plot2 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(log2nor_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# regularized-logarithm transformation (rlog)
colnames(rld_df)[1:2] <- c("x", "y")
plot3 <- ggplot(rld_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_ka_rld.png', sep=""))
plot3 + geom_text(x=4,
                  y=15,
                  label=paste("Pearson = ",
                              round(rld_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

# variance stabilizing transformation (vst)
colnames(vsd_df)[1:2] <- c("x", "y")
plot4 <- ggplot(vsd_df,
                aes(x = x, y = y)) +
                geom_hex(bins = 80) +
                coord_fixed() +
                xlab(colnames(dds)[1]) +
                ylab(colnames(dds)[2])
png(paste('./Outputs/DESeq2/Plots/Correlation/', string2, '_ka_vsd.png', sep=""))
plot4 + geom_text(x=11,
                  y=15,
                  label=paste("Pearson = ",
                              round(vsd_r$estimate,
                                    digits=3)),
                  color = "red",
                  size = 6)
dev.off()

##### 5.2.2.2) Pefanis

##### With FeatureCounts :

In [23]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_pefanis_with_replicates_fc.R SRR1302208 SRR1302283 SRR1302414 SRR1302785 SRR1302786 SRR1302787 SRR1302788 SRR1302789 &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_pefanis_with_replicates_fc.verbose;

Content of deseq2_mutant_vs_wt_pefanis_with_replicates_fc.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

# Open featureCounts results for the same mouse model regardless of the experiment

args = commandArgs(trailingOnly=TRUE)

#TEST#
#args <- c('SRR1302208', 'SRR1302283', 'SRR1302414', 'SRR1302785', 'SRR1302786', 'SRR1302787', 'SRR1302788', 'SRR1302789')

WT1 = read.table(paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep=""))
KO1 = read.table(paste("./Outputs/FeatureCounts/",args[2],"_ensembl_gene_id.tsv", sep=""))
WT2 = read.table(paste("./Outputs/FeatureCounts/",args[3],"_ensembl_gene_id.tsv", sep=""))
KO2 = read.table(paste("./Outputs/FeatureCounts/",args[4],"_ensembl_gene_id.tsv", sep=""))
WT3 = read.table(paste("./Outputs/FeatureCounts/",args[5],"_ensembl_gene_id.tsv", sep=""))
KO3 = read.table(paste("./Outputs/FeatureCounts/",args[6],"_ensembl_gene_id.tsv", sep=""))
WT4 = read.table(paste("./Outputs/FeatureCounts/",args[7],"_ensembl_gene_id.tsv", sep=""))
KO4 = read.table(paste("./Outputs/FeatureCounts/",args[8],"_ensembl_gene_id.tsv", sep=""))

paste("./Outputs/FeatureCounts/",args[1],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[2],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[3],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[4],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[5],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[6],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[7],"_ensembl_gene_id.tsv", sep="")
paste("./Outputs/FeatureCounts/",args[8],"_ensembl_gene_id.tsv", sep="")

df = data.frame(row.names = row.names(WT1),
             WT1 = WT1[1],
             KO1 = KO1[1],
             WT2 = WT2[1],
             KO2 = KO2[1],
             WT3 = WT3[1],
             KO3 = KO3[1],
             WT4 = WT4[1],
             KO4 = KO4[1])

colnames(df) <- c(args[1],args[2],args[3],args[4],args[5],args[6],args[7],args[8])
# Create design
condition <- c('WT', 'KO', 'WT', 'KO', 'WT', 'KO', 'WT', 'KO')
type <- c('paired-end', 'paired-end', 'single-end', 'single-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(args[1],args[2],args[3],args[4],args[5],args[6],args[7],args[8])
all(rownames(df_infos) %in% colnames(df))

# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromMatrix(countData = df,
                              colData = df_infos,
                              design = ~ condition)

# Sanity check
dds

# Pre-filtering removing rows that have only 0 or 1 read
dds <- dds[ rowSums(counts(dds)) > 1, ]
# Check on many genes it discarded
summary(dds)
# Set factor to tell which one is the control group
dds$condition <- factor(dds$condition, levels=c('WT','KO'))

# Start actual DE analysis
dds <- DESeq(dds)
res <- results(dds)
# We are using DESeq2_1.16.1, thus betaPrior is set to 'FALSE' and we need to obtain shrunken LFCs with function ‘lfcShrink’.
# https://support.bioconductor.org/p/95695/
res <- lfcShrink(dds,
       contrast=c("condition","WT","KO"),
       res=res)
summary(res, alpha=0.05)
# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png('./Outputs/DESeq2/Plots/MA/KO_vs_WT_pefanis_dup_fc_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()

# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png('./Outputs/DESeq2/Plots/Volcano/KO_vs_WT_pefanis_dup_fc_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")

sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
# Print sizeFactors for future BAM normalization
sizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]

colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT1_normalized_count"
colnames(nor_counts_df)[3] <- "KO1_normalized_count"
colnames(nor_counts_df)[4] <- "WT2_normalized_count"
colnames(nor_counts_df)[5] <- "KO2_normalized_count"
colnames(nor_counts_df)[6] <- "WT3_normalized_count"
colnames(nor_counts_df)[7] <- "KO3_normalized_count"
colnames(nor_counts_df)[8] <- "WT4_normalized_count"
colnames(nor_counts_df)[9] <- "KO4_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

#Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]

write.table(res_biotyped_counts,
            './Outputs/DESeq2/Raw/KO_vs_WT_pefanis_dup_all_fc.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

#FILTER
res_biotyped_counts<-res_biotyped_counts[complete.cases(res_biotyped_counts$padj),]
res_biotyped_counts<-res_biotyped_counts[(res_biotyped_counts$padj<0.05),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save the result in a TSV
write.table(res_biotyped_counts,
            './Outputs/DESeq2/Filtered/KO_vs_WT_pefanis_dup_fc.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

##### With Kallisto :

In [10]:
### CODE ###
pathway='/run/user/1000/gvfs/smb-share:server=ubox.unilim.fr,share=biscem/BioInformatique/Responsables/Commun/PTCB';
cd $pathway;
Rscript $pathway/Scripts/deseq2_mutant_vs_wt_pefanis_with_replicates_ka.R SRR1302208 SRR1302283 SRR1302414 SRR1302785 SRR1302786 SRR1302787 SRR1302788 SRR1302789 &>> $pathway/Outputs/DESeq2/Verbose/deseq2_mutant_vs_wt_pefanis_with_replicates_ka.verbose;

Content of deseq2_mutant_vs_wt_pefanis_with_replicates_ka.R :

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

library("tximport")
library("biomaRt")
library("GenomicFeatures")

args = commandArgs(trailingOnly=TRUE)

#TEST#
#args <- c('SRR1302208', 'SRR1302283', 'SRR1302414', 'SRR1302785', 'SRR1302786', 'SRR1302787', 'SRR1302788', 'SRR1302789')

WT1 = paste("./Outputs/Kallisto/kallisto",args[1],"/abundance_ensembl_gene_id.tsv", sep="")
KO1 = paste("./Outputs/Kallisto/kallisto",args[2],"/abundance_ensembl_gene_id.tsv", sep="")
WT2 = paste("./Outputs/Kallisto/kallisto",args[3],"/abundance_ensembl_gene_id.tsv", sep="")
KO2 = paste("./Outputs/Kallisto/kallisto",args[4],"/abundance_ensembl_gene_id.tsv", sep="")
WT3 = paste("./Outputs/Kallisto/kallisto",args[5],"/abundance_ensembl_gene_id.tsv", sep="")
KO3 = paste("./Outputs/Kallisto/kallisto",args[6],"/abundance_ensembl_gene_id.tsv", sep="")
WT4 = paste("./Outputs/Kallisto/kallisto",args[7],"/abundance_ensembl_gene_id.tsv", sep="")
KO4 = paste("./Outputs/Kallisto/kallisto",args[8],"/abundance_ensembl_gene_id.tsv", sep="")

WT1
KO1
WT2
KO2
WT3
KO3
WT4
KO4

files <- file.path(c(WT1, KO1, WT2, KO2, WT3, KO3, WT4, KO4))
names(files) <- paste0(c(args[1], args[2], args[3], args[4], args[5], args[6], args[7], args[8]))
files

mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_transcript_id",
                                            "external_gene_name",
                                            "gene_biotype"),
                                            mart = mart)

mmusculusEnsembl <- makeTxDbFromBiomart(dataset="mmusculus_gene_ensembl")

k <- keys(mmusculusEnsembl, keytype = "GENEID")
df <- select(mmusculusEnsembl, keys = k, keytype = "GENEID", columns = "TXNAME")
tx2gene <- df[, 2:1]
txi.kallisto.tsv <- tximport(files, type = "kallisto", tx2gene = tx2gene)
df <- data.frame(txi.kallisto.tsv$counts, check.names = FALSE)

condition <- c('WT', 'KO', 'WT', 'KO', 'WT', 'KO', 'WT', 'KO')
type <- c('paired-end', 'paired-end', 'single-end', 'single-end', 'paired-end', 'paired-end', 'paired-end', 'paired-end')
df_infos <- data.frame(condition, type)
row.names(df_infos) <- c(args[1],args[2],args[3],args[4],args[5],args[6],args[7],args[8])
all(rownames(df_infos) %in% colnames(df))
# Start DESeq2 analysis
library("DESeq2")
dds <- DESeqDataSetFromTximport(txi.kallisto.tsv, df_infos, ~condition) 
dds <- dds[ rowSums(counts(dds)) > 1, ]
dds$condition <- factor(dds$condition, levels=c('WT','KO'))
dds <- DESeq(dds)
res <- results(dds)
res <- lfcShrink(dds, contrast=c("condition","WT","KO"), res=res)

# MA plot : the log2 of ratio of the counts between two conditions is represented as a function of the log2 of the mean of the counts
png('./Outputs/DESeq2/Plots/MA/KO_vs_WT_pefanis_dup_ka_MA.png')
plotMA(res,
       ylim=c(min(res$log2FoldChange),
        max(res$log2FoldChange)))
dev.off()
# Volcano plot : the log10 of the adjusted pvalue is represented as a function of the log2 of ratio of the counts.
png('./Outputs/DESeq2/Plots/Volcano/KO_vs_WT_pefanis_dup_ka_volcano.png')
plot(res$log2FoldChange,
     -log10(res$padj),
     xlim=c(min(res$log2FoldChange), max(res$log2FoldChange)),
     ylim=c(0, 15),
     xlab="log2 fold change",
     ylab="-log10 p-value",
     type="n")

sel <- which(res$log2FoldChange>=1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="red")
sel <- which(res$log2FoldChange<1 & res$log2FoldChange>-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="grey")
sel <- which(res$log2FoldChange<=-1)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="green")
sel <- which(res$padj>0.05)
points(res[sel,"log2FoldChange"], -log10(res[sel,"padj"]),col="black")
abline(v=1,lty=3)
abline(v=-1,lty=3)
dev.off()
# Retrieve normalized counts for each condition for future merge
dds <- estimateSizeFactors(dds)
# Print sizeFactors for future BAM normalization
sizeFactors(dds)
nor_counts <- counts(dds, normalized=TRUE)
nor_counts_df <- as.data.frame(nor_counts)
library(data.table)
nor_counts_df <- setDT(nor_counts_df, keep.rownames = TRUE)[]

colnames(nor_counts_df)[1] <- "ensembl_gene_id"
colnames(nor_counts_df)[2] <- "WT1_normalized_count"
colnames(nor_counts_df)[3] <- "KO1_normalized_count"
colnames(nor_counts_df)[4] <- "WT2_normalized_count"
colnames(nor_counts_df)[5] <- "KO2_normalized_count"
colnames(nor_counts_df)[6] <- "WT3_normalized_count"
colnames(nor_counts_df)[7] <- "KO3_normalized_count"
colnames(nor_counts_df)[8] <- "WT4_normalized_count"
colnames(nor_counts_df)[9] <- "KO4_normalized_count"
# Then tidy = TRUE for compatibility with biomaRt merge
res <- results(dds, tidy = TRUE)
colnames(res)[1] <- "ensembl_gene_id"
# Check
summary(res)
head(res)
# Add biotypes infos
# Get ensembl infos
library("biomaRt")
mart <- biomaRt::useMart(biomart = "ENSEMBL_MART_ENSEMBL",
                        dataset = "mmusculus_gene_ensembl",
                        host = 'ensembl.org')             
mart_infos <- biomaRt::getBM(attributes = c("ensembl_gene_id",
                                            "external_gene_name",
                                            "gene_biotype",
                                            "chromosome_name"),
                                            mart = mart)
# Merge our DE results with ensembl infos for based on same gene_id in column
res_biotyped <- merge(x = res,
                      y = mart_infos,
                      by = "ensembl_gene_id",
                      all.x = TRUE)

####NA removed
res_biotyped$external_gene_name[is.na(res_biotyped$external_gene_name)] <- "None"
res_biotyped$gene_biotype[is.na(res_biotyped$gene_biotype)] <- "None"
res_biotyped$chromosome_name[is.na(res_biotyped$chromosome_name)] <- "None"

# Re-order columns & rename some to make more sense
res_biotyped <- res_biotyped[,c(1,8,10,9,2:7)]
colnames(res_biotyped)[2] <- "ensembl_gene_name"
colnames(res_biotyped)[3] <- "ensembl_chromosome"
colnames(res_biotyped)[4] <- "ensembl_gene_biotype"

# Add normalized counts info
res_biotyped_counts <- merge(x = res_biotyped,
                             y = nor_counts_df,
                             by = "ensembl_gene_id",
                             all.x = TRUE)

#Transform exponential to decimal
options(scipen = 50)

#Remove blood contamination
res_biotyped_counts <- res_biotyped_counts[- grep("Hb[a,q,b]", res_biotyped_counts$ensembl_gene_name),]

write.table(res_biotyped_counts,
            './Outputs/DESeq2/Raw/KO_vs_WT_pefanis_dup_all_ka.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

# Keep Aicda for control
keep_aicda <- res_biotyped_counts[res_biotyped_counts$ensembl_gene_name == 'Aicda',] 

#FILTER
res_biotyped_counts<-res_biotyped_counts[complete.cases(res_biotyped_counts$padj),]
res_biotyped_counts<-res_biotyped_counts[(res_biotyped_counts$padj<0.05),]

# Keep Aicda for control
if(!'Aicda' %in% res_biotyped_counts$ensembl_gene_name){res_biotyped_counts <- rbind(res_biotyped_counts, keep_aicda)}

# Save the result in a TSV
write.table(res_biotyped_counts,
            './Outputs/DESeq2/Filtered/KO_vs_WT_pefanis_dup_ka.tsv',
            append=F, sep='\t',
            col.names=T,
            row.names=F)

### 5.3) Output formatting

##### Point to Comma :

In [ ]:
### CODE ###
mkdir -p Comma;
for f in *.tsv;
do sed 's/\./,/g' $f > Comma/${f%.*}'_comma.tsv';
done;

### 5.4) Genes of interest list for MAR vs WT

List "genes_of_interest_mar.tsv" is constructed from :
- All members of the ARID family
- All members of thee SATB family
- Cux1
- AID off targets genes in Liu et al., Nature Letters, 02/2008, Supp Fig1
- Genes on the first page of Yamane et al., Nature Immunology, 01/2011, S1 Table

In [ ]:
### CODE ###
# On filtered results
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates/Filtered_top_DE;
while read line;
do grep $line MAR_vs_WT_R_YD_top_DE.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv;
# Nothing
while read line;
do grep $line MAR_vs_WT_S_YD_top_DE.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv;
# Nothing

# Given that there was nothing on filtered, at least output those in raw data
cd ~/Desktop/PTCB/BISCEm/Outputs/DESeq2/Mutant_vs_WT_no_replicates;
while read line;
do grep $line MAR_vs_WT_R_YD.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv > ~/Desktop/PTCB/BISCEm/Mar/MAR_vs_WT_R_YD_gon.tsv;
while read line;
do grep $line MAR_vs_WT_S_YD.tsv;
done < ~/Desktop/PTCB/BISCEm/Mar/genes_of_interest_mar.tsv > ~/Desktop/PTCB/BISCEm/Mar/MAR_vs_WT_S_YD_gon.tsv;

### 5.5) Heatmap & Venn diagram

##### Normalized counts heatmap

The goal is to plot all significantly DE genes on a heatmap for all DESeq2 comparisons, even 1vs1 comparisons. The log2 of normalized counts is best suited for this purpose, as explained here : https://support.bioconductor.org/p/76712/. <br>

For instance, consider 3B4_R vs 3B4_S comparison. To obtain a heatmap showing DE genes, we need a variable with values for both R and S. Log2(fold_change) cannot achieve this. Counts can fulfil this task, but we want to normalize them to avoid misinterpretation. It is suggested to use variance stabilized normalized counts (varianceStabilizingTransformation function). Given that the DE genes we consider significant (SDE) are already filtered (on log2(fold_change), p_value and counts), they have robust counts and in those situations, varianceStabilizingTransformation is equivalent to using log2(normalized_count). Thus for the sake of simplicity, we will use log2(normalized_count).

To select SDE which appear on heatmaps & Venn diagrams : <br>
Take the 100 SDE with highest absolute fold change values in featureCounts & in Kallisto results. <br>
Intersect these 2 lists of 100 SDE, yielding a list of X SDE genes common to both featureCounts & Kallisto. These X genes plus aicda (requested by biologist as a control) and minus hemoglobin (Hbq1a, Hbb-y, Hbb-bh1, Hbb-bs, Hba-x, Hba-a2, Hba-a1, Hbq1b, Hbb-bt, Hbb-bh2, Hbb-bh3, Hba-ps4, Hbb-bh0) will appear on the heatmap.

#### 5.5.1) Heatmap & Venn R vs S (1 vs  1)

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

###New method

#Heatmap creation R & S

#To launch from BISCEm/PTCB

args = commandArgs(trailingOnly=TRUE)

#TEST#
#args <- c('3pRR_EP')

library(plyr)

##########FeatureCounts##########

fc = read.table(paste0("./Outputs/DESeq2/Filtered/R_vs_S_",args[1],"_fc.tsv"), header=TRUE, stringsAsFactors=FALSE)
fc <- fc[!duplicated(fc$ensembl_gene_name),]
fc_aicda <- fc[fc$ensembl_gene_name == 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'None',]

##########Kallisto##########

ka = read.table(paste0("./Outputs/DESeq2/Filtered/R_vs_S_",args[1],"_ka.tsv"), header=TRUE, stringsAsFactors=FALSE)
ka <- ka[!duplicated(ka$ensembl_gene_name),]
ka_aicda <- ka[ka$ensembl_gene_name == 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'None',]

paste0("./Outputs/DESeq2/Filtered/R_vs_S_",args[1],"_fc.tsv")

futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")

library("VennDiagram")
venn.diagram(list(featureCounts = fc[,2], Kallisto = ka[,2]), fill = c(2,5), alpha = 0.3, na="remove", filename = paste0('./Outputs/Heatmap/Heatmap_RvS_',args[1],'_VennDiagram.png'), imagetype="png");


fc <- head(fc[order(abs(fc$log2FoldChange), decreasing=TRUE),],100)
ka <- head(ka[order(abs(ka$log2FoldChange), decreasing=TRUE),],100)

fc <- rbind(fc, fc_aicda)
ka <- rbind(ka, ka_aicda)

gene_name_list <- list()
for (line in c(fc[,2], ka[,2])){
	if (is.null(gene_name_list[[line]]) == TRUE){
		gene_name_list[[line]] = 1
	}
	else{
		gene_name_list[[line]] = gene_name_list[[line]]+1
	}
}
i=1
while (i < length(gene_name_list)){
	if (gene_name_list[[i]] < 2){
		gene_name_list[i] <- NULL
		i=i-1
	}
	i=i+1
	if (i == length(gene_name_list)){
		if (gene_name_list[[i]] < 2){
			gene_name_list[i] <- NULL
		}
	}
}

fc <- fc[(fc[,2] %in% names(gene_name_list)),]
ka <- ka[(ka[,2] %in% names(gene_name_list)),]


fc_filtered <- fc[,c("ensembl_gene_id","ensembl_gene_name","R_normalized_count", "S_normalized_count")]
ka_filtered <- ka[,c("ensembl_gene_id","ensembl_gene_name","R_normalized_count", "S_normalized_count")]

fc_filtered[,3] <- log(fc_filtered[,3]+1, 2)
fc_filtered[,4] <- log(fc_filtered[,4]+1, 2)

ka_filtered[,3] <- log(ka_filtered[,3]+1, 2)
ka_filtered[,4] <- log(ka_filtered[,4]+1, 2)


data_frame_fc <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(fc_filtered))
data_frame_ka <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(ka_filtered))

data_frame_fc <- data_frame_fc[complete.cases(data_frame_fc), ]
data_frame_ka <- data_frame_ka[complete.cases(data_frame_ka), ]

row.names(data_frame_fc) <- data_frame_fc$ensembl_gene_name
row.names(data_frame_ka) <- data_frame_ka$ensembl_gene_name

data_frame_fc$ensembl_gene_id <- NULL
data_frame_fc$ensembl_gene_name <- NULL

data_frame_ka$ensembl_gene_id <- NULL
data_frame_ka$ensembl_gene_name <- NULL

data_frame_fc <- data.matrix(data_frame_fc, rownames.force = NA)
data_frame_ka <- data.matrix(data_frame_ka, rownames.force = NA)

library("pheatmap")

paletteLength <- length(data_frame_fc)
myColor <- colorRampPalette(c("white", "blue"))(paletteLength)

pheatmap(data_frame_fc, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_RvS_',args[1],'_fc.png'), main=paste0("Heatmap Resting versus Stimulated for ",args[1]," log2 counts for featureCounts"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
pheatmap(data_frame_ka, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_RvS_',args[1],'_ka.png'), main=paste0("Heatmap Resting versus Stimulated for ",args[1]," log2 counts for kallisto"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
dev.off()
#
#library(Heatmap)
#Heatmap(data_frame_fc, name = "foo", km = 2, row_title_gp = gpar(col = c("red", "blue"), font = 1:2), row_names_gp = gpar(col = c("green", "orange"), fontsize = c(10, 14)))

#### 5.5.2) Heatmap & Venn Mutant vs WT (1 vs 1)

##### 5.5.2.1) CRIBL R

In [2]:
### CODE ###
#!/usr/bin/env Rscript

###New method

#Heatmap creation R & S

#To launch from BISCEm/PTCB

args = commandArgs(trailingOnly=TRUE)

#TEST#
#args <- c('3B4_vs_WT_R_EP')

library(plyr)

##########FeatureCounts##########

fc = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv"), header=TRUE, stringsAsFactors=FALSE)
fc <- fc[!duplicated(fc$ensembl_gene_name),]
fc_aicda <- fc[fc$ensembl_gene_name == 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'None',]

##########Kallisto##########

ka = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_ka.tsv"), header=TRUE, stringsAsFactors=FALSE)
ka <- ka[!duplicated(ka$ensembl_gene_name),]
ka_aicda <- ka[ka$ensembl_gene_name == 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'None',]

paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv")

futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")

library("VennDiagram")
venn.diagram(list(featureCounts = fc[,2], Kallisto = ka[,2]), fill = c(2,5), alpha = 0.3, na="remove", filename = paste0('./Outputs/Heatmap/Heatmap_',args[1],'_VennDiagram.png'), imagetype="png");

fc <- head(fc[order(abs(fc$log2FoldChange), decreasing=TRUE),],100)
ka <- head(ka[order(abs(ka$log2FoldChange), decreasing=TRUE),],100)

fc <- rbind(fc, fc_aicda)
ka <- rbind(ka, ka_aicda)

gene_name_list <- list()
for (line in c(fc[,2], ka[,2])){
	if (is.null(gene_name_list[[line]]) == TRUE){
		gene_name_list[[line]] = 1
	}
	else{
		gene_name_list[[line]] = gene_name_list[[line]]+1
	}
}
i=1
while (i < length(gene_name_list)){
	if (gene_name_list[[i]] < 2){
		gene_name_list[i] <- NULL
		i=i-1
	}
	i=i+1
	if (i == length(gene_name_list)){
		if (gene_name_list[[i]] < 2){
			gene_name_list[i] <- NULL
		}
	}
}

fc <- fc[(fc[,2] %in% names(gene_name_list)),]
ka <- ka[(ka[,2] %in% names(gene_name_list)),]


fc_filtered <- fc[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]
ka_filtered <- ka[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]

fc_filtered[,3] <- log(fc_filtered[,3]+1, 2)
fc_filtered[,4] <- log(fc_filtered[,4]+1, 2)

ka_filtered[,3] <- log(ka_filtered[,3]+1, 2)
ka_filtered[,4] <- log(ka_filtered[,4]+1, 2)


data_frame_fc <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(fc_filtered))
data_frame_ka <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(ka_filtered))

data_frame_fc <- data_frame_fc[complete.cases(data_frame_fc), ]
data_frame_ka <- data_frame_ka[complete.cases(data_frame_ka), ]

row.names(data_frame_fc) <- data_frame_fc$ensembl_gene_name
row.names(data_frame_ka) <- data_frame_ka$ensembl_gene_name

data_frame_fc$ensembl_gene_id <- NULL
data_frame_fc$ensembl_gene_name <- NULL

data_frame_ka$ensembl_gene_id <- NULL
data_frame_ka$ensembl_gene_name <- NULL

data_frame_fc <- data.matrix(data_frame_fc, rownames.force = NA)
data_frame_ka <- data.matrix(data_frame_ka, rownames.force = NA)

library("pheatmap")

paletteLength <- length(data_frame_fc)
myColor <- colorRampPalette(c("white", "blue"))(paletteLength)

pheatmap(data_frame_fc, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_fc.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Resting log2 counts for featureCounts"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
pheatmap(data_frame_ka, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_ka.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Resting log2 counts for kallisto"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
dev.off()

##### 5.5.2.2) CRIBL S

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

###New method

#Heatmap creation R & S

#To launch from BISCEm/PTCB

args = commandArgs(trailingOnly=TRUE)

#TEST#
#args <- c('3B4_vs_WT_S_EP')

library(plyr)

##########FeatureCounts##########

fc = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv"), header=TRUE, stringsAsFactors=FALSE)
fc <- fc[!duplicated(fc$ensembl_gene_name),]
fc_aicda <- fc[fc$ensembl_gene_name == 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'None',]

##########Kallisto##########

ka = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_ka.tsv"), header=TRUE, stringsAsFactors=FALSE)
ka <- ka[!duplicated(ka$ensembl_gene_name),]
ka_aicda <- ka[ka$ensembl_gene_name == 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'None',]

paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv")

futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")

library("VennDiagram")
venn.diagram(list(featureCounts = fc[,2], Kallisto = ka[,2]), fill = c(2,5), alpha = 0.3, na="remove", filename = paste0('./Outputs/Heatmap/Heatmap_',args[1],'_VennDiagram.png'), imagetype="png");

fc <- head(fc[order(abs(fc$log2FoldChange), decreasing=TRUE),],100)
ka <- head(ka[order(abs(ka$log2FoldChange), decreasing=TRUE),],100)

fc <- rbind(fc, fc_aicda)
ka <- rbind(ka, ka_aicda)

gene_name_list <- list()
for (line in c(fc[,2], ka[,2])){
	if (is.null(gene_name_list[[line]]) == TRUE){
		gene_name_list[[line]] = 1
	}
	else{
		gene_name_list[[line]] = gene_name_list[[line]]+1
	}
}
i=1
while (i < length(gene_name_list)){
	if (gene_name_list[[i]] < 2){
		gene_name_list[i] <- NULL
		i=i-1
	}
	i=i+1
	if (i == length(gene_name_list)){
		if (gene_name_list[[i]] < 2){
			gene_name_list[i] <- NULL
		}
	}
}

fc <- fc[(fc[,2] %in% names(gene_name_list)),]
ka <- ka[(ka[,2] %in% names(gene_name_list)),]


fc_filtered <- fc[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]
ka_filtered <- ka[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]

fc_filtered[,3] <- log(fc_filtered[,3]+1, 2)
fc_filtered[,4] <- log(fc_filtered[,4]+1, 2)

ka_filtered[,3] <- log(ka_filtered[,3]+1, 2)
ka_filtered[,4] <- log(ka_filtered[,4]+1, 2)


data_frame_fc <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(fc_filtered))
data_frame_ka <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(ka_filtered))

data_frame_fc <- data_frame_fc[complete.cases(data_frame_fc), ]
data_frame_ka <- data_frame_ka[complete.cases(data_frame_ka), ]

row.names(data_frame_fc) <- data_frame_fc$ensembl_gene_name
row.names(data_frame_ka) <- data_frame_ka$ensembl_gene_name

data_frame_fc$ensembl_gene_id <- NULL
data_frame_fc$ensembl_gene_name <- NULL

data_frame_ka$ensembl_gene_id <- NULL
data_frame_ka$ensembl_gene_name <- NULL

data_frame_fc <- data.matrix(data_frame_fc, rownames.force = NA)
data_frame_ka <- data.matrix(data_frame_ka, rownames.force = NA)

library("pheatmap")

paletteLength <- length(data_frame_fc)
myColor <- colorRampPalette(c("white", "blue"))(paletteLength)

pheatmap(data_frame_fc, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_fc.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Stimulated log2 counts for featureCounts"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
pheatmap(data_frame_ka, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_ka.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Stimulated log2 counts for kallisto"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
dev.off()

##### 5.5.2.2) Pefanis S

In [ ]:
### CODE ###
#!/usr/bin/env Rscript

###New method

#Heatmap creation R & S

#To launch from BISCEm/PTCB

args = commandArgs(trailingOnly=TRUE)

#TEST#
args <- c('KO_vs_WT_pefanis_dup')

library(plyr)

##########FeatureCounts##########

fc = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv"), header=TRUE, stringsAsFactors=FALSE)
fc <- fc[!duplicated(fc$ensembl_gene_name),]
fc_aicda <- fc[fc$ensembl_gene_name == 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'Aicda',]
fc <- fc[fc$ensembl_gene_name != 'None',]

##########Kallisto##########

ka = read.table(paste0("./Outputs/DESeq2/Filtered/",args[1],"_ka.tsv"), header=TRUE, stringsAsFactors=FALSE)
ka <- ka[!duplicated(ka$ensembl_gene_name),]
ka_aicda <- ka[ka$ensembl_gene_name == 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'Aicda',]
ka <- ka[ka$ensembl_gene_name != 'None',]

paste0("./Outputs/DESeq2/Filtered/",args[1],"_fc.tsv")

futile.logger::flog.threshold(futile.logger::ERROR, name = "VennDiagramLogger")

library("VennDiagram")
venn.diagram(list(featureCounts = fc[,2], Kallisto = ka[,2]), fill = c(2,5), alpha = 0.3, na="remove", filename = paste0('./Outputs/Heatmap/Heatmap_',args[1],'_VennDiagram.png'), imagetype="png");

fc <- head(fc[order(abs(fc$log2FoldChange), decreasing=TRUE),],100)
ka <- head(ka[order(abs(ka$log2FoldChange), decreasing=TRUE),],100)

fc <- rbind(fc, fc_aicda)
ka <- rbind(ka, ka_aicda)

gene_name_list <- list()
for (line in c(fc[,2], ka[,2])){
	if (is.null(gene_name_list[[line]]) == TRUE){
		gene_name_list[[line]] = 1
	}
	else{
		gene_name_list[[line]] = gene_name_list[[line]]+1
	}
}
i=1
while (i < length(gene_name_list)){
	if (gene_name_list[[i]] < 2){
		gene_name_list[i] <- NULL
		i=i-1
	}
	i=i+1
	if (i == length(gene_name_list)){
		if (gene_name_list[[i]] < 2){
			gene_name_list[i] <- NULL
		}
	}
}

fc <- fc[(fc[,2] %in% names(gene_name_list)),]
ka <- ka[(ka[,2] %in% names(gene_name_list)),]

###MEAN OF COLUMNS
fc$WT_normalized_count <- rowMeans(fc[,c("WT1_normalized_count","WT2_normalized_count","WT3_normalized_count", "WT4_normalized_count")])
fc$KO_normalized_count <- rowMeans(fc[,c("KO1_normalized_count","KO2_normalized_count","KO3_normalized_count", "KO4_normalized_count")])
fc$WT1_normalized_count <- NULL
fc$WT2_normalized_count <- NULL
fc$WT3_normalized_count <- NULL
fc$WT4_normalized_count <- NULL
fc$KO1_normalized_count <- NULL
fc$KO2_normalized_count <- NULL
fc$KO3_normalized_count <- NULL
fc$KO4_normalized_count <- NULL

ka$WT_normalized_count <- rowMeans(ka[,c("WT1_normalized_count","WT2_normalized_count","WT3_normalized_count", "WT4_normalized_count")])
ka$KO_normalized_count <- rowMeans(ka[,c("KO1_normalized_count","KO2_normalized_count","KO3_normalized_count", "KO4_normalized_count")])
ka$WT1_normalized_count <- NULL
ka$WT2_normalized_count <- NULL
ka$WT3_normalized_count <- NULL
ka$WT4_normalized_count <- NULL
ka$KO1_normalized_count <- NULL
ka$KO2_normalized_count <- NULL
ka$KO3_normalized_count <- NULL
ka$KO4_normalized_count <- NULL
#################é

fc_filtered <- fc[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]
ka_filtered <- ka[,c("ensembl_gene_id","ensembl_gene_name","WT_normalized_count", "KO_normalized_count")]

fc_filtered[,3] <- log(fc_filtered[,3]+1, 2)
fc_filtered[,4] <- log(fc_filtered[,4]+1, 2)

ka_filtered[,3] <- log(ka_filtered[,3]+1, 2)
ka_filtered[,4] <- log(ka_filtered[,4]+1, 2)


data_frame_fc <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(fc_filtered))
data_frame_ka <- Reduce(function(x, y) merge(x, y, by=c("ensembl_gene_id", "ensembl_gene_name"), all=TRUE, stringsAsFactors=FALSE), list(ka_filtered))

data_frame_fc <- data_frame_fc[complete.cases(data_frame_fc), ]
data_frame_ka <- data_frame_ka[complete.cases(data_frame_ka), ]

row.names(data_frame_fc) <- data_frame_fc$ensembl_gene_name
row.names(data_frame_ka) <- data_frame_ka$ensembl_gene_name

data_frame_fc$ensembl_gene_id <- NULL
data_frame_fc$ensembl_gene_name <- NULL

data_frame_ka$ensembl_gene_id <- NULL
data_frame_ka$ensembl_gene_name <- NULL

data_frame_fc <- data.matrix(data_frame_fc, rownames.force = NA)
data_frame_ka <- data.matrix(data_frame_ka, rownames.force = NA)

library("pheatmap")

paletteLength <- length(data_frame_fc)
myColor <- colorRampPalette(c("white", "blue"))(paletteLength)

pheatmap(data_frame_fc, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_fc.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Stimulated log2 counts for featureCounts"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
pheatmap(data_frame_ka, color=myColor, filename=paste0('./Outputs/Heatmap/Heatmap_',args[1],'_ka.png'), main=paste0("Heatmap",strsplit(args[1], "_")[[1]][1]," versus WT for Stimulated log2 counts for kallisto"), cluster_cols=TRUE, cluster_rows=TRUE, height=10, width=10)
dev.off()

#### 5.5.3) Heatmap & Venn (X vs X)

Only if Sandrine wants them again.

### 5.6) Normalized visualisation

After DE genes have been called, biologists want to go back to "the gene level" and visualize said DE genes in a genome browser, with associated read pileup / coverage for each condition. <br>

Two things have to be considered here : <br>
- Direct visualization from raw BAMs is skewed, because DESeq2 called the DE genes based on normalized counts, not raw counts. Thus we need to "normalize the BAM". This is achieved by retrieving for each DESeq2 comparision the result of the "sizeFactors(dds)" command.  <br>
See p5 of https://bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf : "whether or not to divide the counts by the size factors".  <br>
An example : Consider the 1vs1 comparison of WT_R_EP vs WT_S_EP. Associated size factors are 0.8560343 (R) and 1.1681774 (S). In the DESeq2 output, take a random result : "ENSMUSG00000000078", which have associated normalized counts : 10241.41 (R) & 925.37 (S). Now check that there were indeed obtained by dividing raw featureCounts counts by size factor : Count value for R : 8767, divided by 0.8560343 = 10241.41 & count value for S : 1081, divided by 1.1681774 = 925.37. <br>


- Our data is paired and strand-specific, thus we can and should split BAM for the read pairs on the forward and on the reverse strand. This is achieved with the "filterRNAstrand" option within the bamCoverage software.

In [ ]:
### CODE ###
# Set paths & folders for normalized BAMs creation
biscem='/home/erwann/Desktop/PTCB/BISCEm';
mkdir -p $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates;
mkdir -p $biscem/Outputs/Normalized_BAM/Verbose;
# mkdir -p $biscem/Outputs/Normalized_BAM/R_vs_S_with_replicates;
# mkdir -p $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_with_replicates;

##################################
### Mutant_vs_WT_no_replicates ###
##################################
# Set scales for Mutant_vs_WT_no_replicates
scale_WT_3B4_R_EP=0.8551271; 
scale_3B4_R_EP=1.1694168;
scale_WT_3B4_S_EP=1.0536938;
scale_3B4_S_EP=0.9490423;
scale_WT_3pRR_R_EP=0.8129696;
scale_3pRR_R_EP=1.2300583;
scale_WT_3pRR_S_EP=1.0164029;
scale_3pRR_S_EP=0.9838618;
scale_WT_DEL_R_EP=0.8951107;
scale_DEL_R_EP=1.1171802;
scale_WT_DEL_S_EP=1.0173134;
scale_DEL_S_EP=0.9829813;
scale_WT_IRIS_R_EP=0.8257228;
scale_IRIS_R_EP=1.2110601;
scale_WT_IRIS_S_EP=0.9916549;
scale_IRIS_S_EP=1.0084153;
scale_WT_PAL_R_EP=0.8831167;
scale_PAL_R_EP=1.1323531;
scale_WT_PAL_S_EP=0.9969528;
scale_PAL_S_EP=1.0030566;
scale_WT_3pRR_R_YD=0.9364606;
scale_3pRR_R_YD=1.0678505;
scale_WT_3pRR_S_YD=1.4142136;
scale_3pRR_S_YD=0.7071068;
scale_WT_EU_R_YD=1.1547005;
scale_EU_R_YD=0.8660254;
scale_WT_EU_S_YD=1.3740364;
scale_EU_S_YD=0.7277828;
scale_WT_MAR_R_YD=0.8312094;
scale_MAR_R_YD=1.2030663;
scale_WT_MAR_S_YD=1.544631;
scale_MAR_S_YD=0.647404;
# Transform scales to account for the fact that we need to divide the counts by the size factors, not multiply
scale_div_WT_3B4_R_EP=$(awk -v scale_WT_3B4_R_EP=$scale_WT_3B4_R_EP 'BEGIN { print (1 / scale_WT_3B4_R_EP) }');
scale_div_3B4_R_EP=$(awk -v scale_3B4_R_EP=$scale_3B4_R_EP 'BEGIN { print (1 / scale_3B4_R_EP) }');
scale_div_WT_3B4_S_EP=$(awk -v scale_WT_3B4_S_EP=$scale_WT_3B4_S_EP 'BEGIN { print (1 / scale_WT_3B4_S_EP) }');
scale_div_3B4_S_EP=$(awk -v scale_3B4_S_EP=$scale_3B4_S_EP 'BEGIN { print (1 / scale_3B4_S_EP) }');
scale_div_WT_3pRR_R_EP=$(awk -v scale_WT_3pRR_R_EP=$scale_WT_3pRR_R_EP 'BEGIN { print (1 / scale_WT_3pRR_R_EP) }');
scale_div_3pRR_R_EP=$(awk -v scale_3pRR_R_EP=$scale_3pRR_R_EP 'BEGIN { print (1 / scale_3pRR_R_EP) }');
scale_div_WT_3pRR_S_EP=$(awk -v scale_WT_3pRR_S_EP=$scale_WT_3pRR_S_EP 'BEGIN { print (1 / scale_WT_3pRR_S_EP) }');
scale_div_3pRR_S_EP=$(awk -v scale_3pRR_S_EP=$scale_3pRR_S_EP 'BEGIN { print (1 / scale_3pRR_S_EP) }');
scale_div_WT_DEL_R_EP=$(awk -v scale_WT_DEL_R_EP=$scale_WT_DEL_R_EP 'BEGIN { print (1 / scale_WT_DEL_R_EP) }');
scale_div_DEL_R_EP=$(awk -v scale_DEL_R_EP=$scale_DEL_R_EP 'BEGIN { print (1 / scale_DEL_R_EP) }');
scale_div_WT_DEL_S_EP=$(awk -v scale_WT_DEL_S_EP=$scale_WT_DEL_S_EP 'BEGIN { print (1 / scale_WT_DEL_S_EP) }');
scale_div_DEL_S_EP=$(awk -v scale_DEL_S_EP=$scale_DEL_S_EP 'BEGIN { print (1 / scale_DEL_S_EP) }');
scale_div_WT_IRIS_R_EP=$(awk -v scale_WT_IRIS_R_EP=$scale_WT_IRIS_R_EP 'BEGIN { print (1 / scale_WT_IRIS_R_EP) }');
scale_div_IRIS_R_EP=$(awk -v scale_IRIS_R_EP=$scale_IRIS_R_EP 'BEGIN { print (1 / scale_IRIS_R_EP) }');
scale_div_WT_IRIS_S_EP=$(awk -v scale_WT_IRIS_S_EP=$scale_WT_IRIS_S_EP 'BEGIN { print (1 / scale_WT_IRIS_S_EP) }');
scale_div_IRIS_S_EP=$(awk -v scale_IRIS_S_EP=$scale_IRIS_S_EP 'BEGIN { print (1 / scale_IRIS_S_EP) }');
scale_div_WT_PAL_R_EP=$(awk -v scale_WT_PAL_R_EP=$scale_WT_PAL_R_EP 'BEGIN { print (1 / scale_WT_PAL_R_EP) }');
scale_div_PAL_R_EP=$(awk -v scale_PAL_R_EP=$scale_PAL_R_EP 'BEGIN { print (1 / scale_PAL_R_EP) }');
scale_div_WT_PAL_S_EP=$(awk -v scale_WT_PAL_S_EP=$scale_WT_PAL_S_EP 'BEGIN { print (1 / scale_WT_PAL_S_EP) }');
scale_div_PAL_S_EP=$(awk -v scale_PAL_S_EP=$scale_PAL_S_EP 'BEGIN { print (1 / scale_PAL_S_EP) }');
scale_div_WT_3pRR_R_YD=$(awk -v scale_WT_3pRR_R_YD=$scale_WT_3pRR_R_YD 'BEGIN { print (1 / scale_WT_3pRR_R_YD) }');
scale_div_3pRR_R_YD=$(awk -v scale_3pRR_R_YD=$scale_3pRR_R_YD 'BEGIN { print (1 / scale_3pRR_R_YD) }');
scale_div_WT_3pRR_S_YD=$(awk -v scale_WT_3pRR_S_YD=$scale_WT_3pRR_S_YD 'BEGIN { print (1 / scale_WT_3pRR_S_YD) }');
scale_div_3pRR_S_YD=$(awk -v scale_3pRR_S_YD=$scale_3pRR_S_YD 'BEGIN { print (1 / scale_3pRR_S_YD) }');
scale_div_WT_EU_R_YD=$(awk -v scale_WT_EU_R_YD=$scale_WT_EU_R_YD 'BEGIN { print (1 / scale_WT_EU_R_YD) }');
scale_div_EU_R_YD=$(awk -v scale_EU_R_YD=$scale_EU_R_YD 'BEGIN { print (1 / scale_EU_R_YD) }');
scale_div_WT_EU_S_YD=$(awk -v scale_WT_EU_S_YD=$scale_WT_EU_S_YD 'BEGIN { print (1 / scale_WT_EU_S_YD) }');
scale_div_EU_S_YD=$(awk -v scale_EU_S_YD=$scale_EU_S_YD 'BEGIN { print (1 / scale_EU_S_YD) }');
scale_div_WT_MAR_R_YD=$(awk -v scale_WT_MAR_R_YD=$scale_WT_MAR_R_YD 'BEGIN { print (1 / scale_WT_MAR_R_YD) }');
scale_div_MAR_R_YD=$(awk -v scale_MAR_R_YD=$scale_MAR_R_YD 'BEGIN { print (1 / scale_MAR_R_YD) }');
scale_div_WT_MAR_S_YD=$(awk -v scale_WT_MAR_S_YD=$scale_WT_MAR_S_YD 'BEGIN { print (1 / scale_WT_MAR_S_YD) }');
scale_div_MAR_S_YD=$(awk -v scale_MAR_S_YD=$scale_MAR_S_YD 'BEGIN { print (1 / scale_MAR_S_YD) }');

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
#for id in '3B4_R_EP' '3B4_S_EP' '3pRR_R_EP' '3pRR_S_EP' 'DEL_R_EP' 'DEL_S_EP' 'IRIS_R_EP' 'IRIS_S_EP' 'PAL_R_EP' 'PAL_S_EP' '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD';
for id in '3pRR_R_YD' '3pRR_S_YD' 'EU_R_YD' 'EU_S_YD' 'MAR_R_YD' 'MAR_S_YD';
do scale=$(eval printf "$""scale_div_"$id);
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/$id"_forward_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $id.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/$id"_reverse_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   scale=$(eval printf "$""scale_div_WT_"$id);
   mutant=${id%%_*};
   wt_file=$(echo $id | sed 's/'$mutant'/WT/g');
   bamCoverage --bam $wt_file.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/"WT_"$id"_forward_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $wt_file.bam \
               --outFileName $biscem/Outputs/Normalized_BAM/Mutant_vs_WT_no_replicates/"WT_"$id"_reverse_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/mutant_vs_wt_no_replicates_bigwig.verbose;

############################
### R_vs_S_no_replicates ###
############################
# Set scales for R_vs_S_no_replicates
scale_3B4_R_EP=1.0019940;
scale_3B4_S_EP=0.9980099;
scale_3pRR_R_EP=1.0258757;
scale_3pRR_S_EP=0.9747769;
scale_DEL_R_EP=0.9876488;
scale_DEL_S_EP=1.0125057;
scale_IRIS_R_EP=1;
scale_IRIS_S_EP=1;
scale_PAL_R_EP=0.9251799;
scale_PAL_S_EP=1.0808709;
scale_WT_R_EP=0.8560343;
scale_WT_S_EP=1.1681774;
scale_3pRR_R_YD=1.4699883;
scale_3pRR_S_YD=0.6802775;
scale_EU_R_YD=1.0939116;
scale_EU_S_YD=0.9141506;
scale_MAR_R_YD=1.7791104;
scale_MAR_S_YD=0.5620787;
scale_WT_R_YD=0.9906975;
scale_WT_S_YD=1.0093899;
# Transform scales to account for the fact that we need to divide the counts by the size factors, not multiply
scale_div_3B4_R_EP=$(awk -v scale_3B4_R_EP=$scale_3B4_R_EP 'BEGIN { print (1 / scale_3B4_R_EP) }');
scale_div_3B4_S_EP=$(awk -v scale_3B4_S_EP=$scale_3B4_S_EP 'BEGIN { print (1 / scale_3B4_S_EP) }');
scale_div_3pRR_R_EP=$(awk -v scale_3pRR_R_EP=$scale_3pRR_R_EP 'BEGIN { print (1 / scale_3pRR_R_EP) }');
scale_div_3pRR_S_EP=$(awk -v scale_3pRR_S_EP=$scale_3pRR_S_EP 'BEGIN { print (1 / scale_3pRR_S_EP) }');
scale_div_DEL_R_EP=$(awk -v scale_DEL_R_EP=$scale_DEL_R_EP 'BEGIN { print (1 / scale_DEL_R_EP) }');
scale_div_DEL_S_EP=$(awk -v scale_DEL_S_EP=$scale_DEL_S_EP 'BEGIN { print (1 / scale_DEL_S_EP) }');
scale_div_IRIS_R_EP=$(awk -v scale_IRIS_R_EP=$scale_IRIS_R_EP 'BEGIN { print (1 / scale_IRIS_R_EP) }');
scale_div_IRIS_S_EP=$(awk -v scale_IRIS_S_EP=$scale_IRIS_S_EP 'BEGIN { print (1 / scale_IRIS_S_EP) }');
scale_div_PAL_R_EP=$(awk -v scale_PAL_R_EP=$scale_PAL_R_EP 'BEGIN { print (1 / scale_PAL_R_EP) }');
scale_div_PAL_S_EP=$(awk -v scale_PAL_S_EP=$scale_PAL_S_EP 'BEGIN { print (1 / scale_PAL_S_EP) }');
scale_div_WT_R_EP=$(awk -v scale_WT_R_EP=$scale_WT_R_EP 'BEGIN { print (1 / scale_WT_R_EP) }');
scale_div_WT_S_EP=$(awk -v scale_WT_S_EP=$scale_WT_S_EP 'BEGIN { print (1 / scale_WT_S_EP) }');
scale_div_3pRR_R_YD=$(awk -v scale_3pRR_R_YD=$scale_3pRR_R_YD 'BEGIN { print (1 / scale_3pRR_R_YD) }');
scale_div_3pRR_S_YD=$(awk -v scale_3pRR_S_YD=$scale_3pRR_S_YD 'BEGIN { print (1 / scale_3pRR_S_YD) }');
scale_div_EU_R_YD=$(awk -v scale_EU_R_YD=$scale_EU_R_YD 'BEGIN { print (1 / scale_EU_R_YD) }');
scale_div_EU_S_YD=$(awk -v scale_EU_S_YD=$scale_EU_S_YD 'BEGIN { print (1 / scale_EU_S_YD) }');
scale_div_MAR_R_YD=$(awk -v scale_MAR_R_YD=$scale_MAR_R_YD 'BEGIN { print (1 / scale_MAR_R_YD) }');
scale_div_MAR_S_YD=$(awk -v scale_MAR_S_YD=$scale_MAR_S_YD 'BEGIN { print (1 / scale_MAR_S_YD) }');
scale_div_WT_R_YD=$(awk -v scale_WT_R_YD=$scale_WT_R_YD 'BEGIN { print (1 / scale_WT_R_YD) }');
scale_div_WT_S_YD=$(awk -v scale_WT_S_YD=$scale_WT_S_YD 'BEGIN { print (1 / scale_WT_S_YD) }');

# Proceed BAMs with bamCoverage
cd $biscem/Outputs/Mapped_vs_GRCm38.p5;
for f in WT*EP*.bam WT*YD*.bam 3B4*.bam 3pRR*EP*.bam 3pRR*YD*.bam DEL*.bam EU*.bam IRIS*.bam MAR*.bam PAL*.bam;
do scale=$(eval printf "$""scale_div_"${f%.*});
   bamCoverage --bam $f \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates/${f%.*}"_forward_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand forward \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
   bamCoverage --bam $f \
               --outFileName $biscem/Outputs/Normalized_BAM/R_vs_S_no_replicates/${f%.*}"_reverse_strand.bigwig" \
               --outFileFormat bigwig \
               --scaleFactor $scale \
               --filterRNAstrand reverse \
               --binSize 1 \
               --numberOfProcessors max \
               --verbose;
done &>> $biscem/Outputs/Normalized_BAM/Verbose/r_vs_s_no_replicates_bigwig.verbose;

### 5.7) Antisense transcription

IGV manual inspection

UClncR

### 5.8) Replicates reproducibility

Correlation plots in DE analysis. <br>
WT vs WT (Eric, Yves), 3pRR vs 3pRR (Eric, Yves), WT vs WT (Pefanis) & compare SDE list vs Mutant vs WT SDE. <br>
Results : https://docs.google.com/spreadsheets/d/14RrurdAtQKyp_o8ILa5fbT40dCva4LLS1ZQQKribFNg/edit?usp=sharing


Comparaison --- Nombres de gènes SDE <br>
Limoges:3pRR_R_Eric_vs_Yves --- 317 <br>
Limoges:3pRR_S_Eric_vs_Yves --- 125 <br>
Limoges:WT_R_Eric_vs_Yves --- 103 <br>
Limoges:WT_S_Eric_vs_Yves --- 193 <br>
Pefanis:SRR1302208/SRR1302414 --- 8786 <br>
Pefanis:SRR1302208/SRR1302786 --- 8885 <br>
Pefanis:SRR1302208/SRR1302788 --- 7955 <br>
Pefanis:SRR1302414/SRR1302786 --- 16443 <br>
Pefanis:SRR1302414/SRR1302788 --- 15674 <br>
Pefanis:SRR1302786/SRR1302788 --- 2907

"Lors de la réunion "trieur", il avait été suggéré pour vérifier la reproductibilité des réplicats de faire, en plus des graphiques de corrélation, l'intersection des listes de gènes significativement différentiellement exprimés (DE) pour les comparaisons entre échantillons considérés comme réplicats (WT_R, WT_S, 3pRR_R, 3pRR_S, présent chez Eric et chez Yves) versus les listes de gènes DE des comparaisons Mutant_vs_WT."


"Voici ce que je ferai <br>
Comparer WT R EP et WT R YD, cela va sortir un liste de gènes DE (liste 1) <br>
Puis comparer WTR EP et hs3b4 R EP, cela sortir une autre liste de gènes DE (liste 2) <br>
Enfin comparer la liste 1 et la liste 2 <br>
Refaire la même chose en changeant le mutant <br>
Est ce plus clair? <br>
Sandrine"